# Eir v1.2 - Automated Bond Valence Sum Analysis

## Overview

Eir performs automated Bond Valence Sum (BVS) analysis on multiple crystal structures from:
- **ICSD CIF files** (downloaded from ICSD database)
- **GSAS-II refinement outputs** (.lst files)

The code uses proper crystallographic symmetry operations via pymatgen to correctly handle all space groups, particularly high-symmetry cases (R-3m, P63/mmc) common in layered oxides.

### 1. Config File Support
- Load oxidation states from `config.json` or Python dictionary
- Enables "Run All" batch processing without manual input
- Automatic fallback to interactive mode if no config provided

### 2. 3D Visualisation
- Automatic visualisation of problematic sites (BVS deviation > 30%)
- Uses py3Dmol for interactive bond visualisation in Jupyter
- Helps diagnose coordination issues (missing bonds, wrong oxidation state)

### 3. Smart Gap Detection
- Adaptive thresholding for Jahn-Teller distorted sites (Mn³⁺, Cu²⁺)
- Considers both absolute gap size and relative gap ratio
- More robust for highly distorted coordination environments

## Key Features

- Automatic coordination number (CN) detection using adaptive gap analysis
- Mixed/partial occupancy support (e.g., Cr₀.₈Ti₀.₂ on same site)
- Bond Valence Sum (BVS) calculation using Gagné & Hawthorne (2015) parameters
- Global Instability Index (GII) calculation for structural stability assessment
- Batch processing of multiple structures with summary CSV output
- Interactive 3D visualisation of problematic sites

## Input Files

### Option 1: ICSD CIF Files
- Place CIF files in a folder
- Filenames should contain CollCode (e.g., `YourCustomFileName_CollCode116507.cif`)
- Script will extract CollCode from filename

### Option 2: GSAS-II Files
- Requires both `.lst` file (refinement output) and corresponding `.cif` file
- Place in same directory with matching base names

## How to Use

### Quick Start (with config file)
```python
# Create config.json in your CIF directory:
{
  "Na": 1,
  "Ni": 2,
  "Mn": 4,
  "O": -2
}

# Then run all cells - walks away for coffee
```

### Interactive Mode (no config)
```python
# Just run all cells
# Script will prompt you for oxidation states
# Useful when analyzing unfamiliar materials
```

### With 3D Visualisation
```python
# Problematic sites (|deviation| > 30%) are automatically visualized
# Look for:
#   - Missing bonds (coordination too low)
#   - Wrong oxidation state assignment
#   - Jahn-Teller distortion
```

## Output Columns

- **File**: Input filename
- **CollCode/Phase**: ICSD CollCode or phase name
- **Space_Group**: Crystallographic space group
- **Element**: Chemical element analyzed
- **Site_Label**: Crystallographic site label
- **CN**: Coordination number (from adaptive gap analysis)
- **BVS**: Bond Valence Sum
- **Formal_Charge**: Expected oxidation state
- **Deviation**: BVS - Formal_Charge
- **GII**: Global Instability Index (whole structure)

## Interpreting Results

### Coordination Numbers (CN)
- **CN = 6**: Octahedral coordination (expected for most transition metals in layered oxides)
- **CN = 4**: Tetrahedral coordination
- **CN = 1-3**: Likely indicates structural problems or missing bonds

### Bond Valence Sum (BVS)
- Should approximately equal the formal oxidation state
- **Deviation < ±0.2 v.u.**: Likely well-bonded
- **Deviation > ±0.3 v.u.**: Over-bonded (negative) or under-bonded (positive)
- **Deviation > ±0.5 v.u.**: Severe problem - check 3D visualisation

### Global Instability Index (GII)
- **GII < 0.20 v.u.**: Stable, well-ordered structure
- **0.20 < GII < 0.40 v.u.**: Moderate strain, possibly acceptable
- **GII > 0.40 v.u.**: Significant structural instability
- **GII > 0.60 v.u.**: Severe problems - synthesis likely difficult

**Note:** The traditional 0.2 v.u. threshold is system-dependent (see Miller & Rondinelli, APL Mater. 2023)

## Common Issues

### Problem: All structures show CN = 1-2 instead of 6
- **Solution**: Already fixed in v1.0 using proper pymatgen symmetry operations

### Problem: "No bond valence parameters found"
- **Cause**: Element/oxidation state combination not in Gagné & Hawthorne database
- **Solution**: Check oxidation state assignment or add parameters manually

### Problem: Very high GII (>1.0) for literature structures
- **Possible causes:**
  - Mixed-phase sample (XRD refinement artefact)
  - Metastable structure
  - Genuine severe strain (synthesis should be difficult)
  
### Problem: High deviation but coordination looks correct
- **Solution**: Check 3D visualisation - may be Jahn-Teller distortion or wrong oxidation state

## Technical Details

### Adaptive Gap Detection (New in v1.1)
- Considers both absolute gap size (default 0.05 Å minimum)
- Also considers relative gap ratio (gap/bond length)
- For Jahn-Teller ions (Mn³⁺, Cu²⁺), uses more lenient thresholds
- Prevents over-splitting of distorted but chemically valid coordination spheres

### 3D Visualisation (New in v1.1)
- Automatically triggers for sites with |deviation| > 30%
- Shows central atom, coordinating atoms, and bonds
- Bond colours indicate bond valence contribution
- Interactive rotation and zooming in Jupyter

## Citation

If using this code, please cite:
- **BVS parameters:** Gagné, O.C. & Hawthorne, F.C. (2015) *Acta Cryst. B* **71**, 562-578
- **Structural validation:** Brown, I.D. (2002) *The Chemical Bond in Inorganic Chemistry: The Bond Valence Model*

## Version History

- **v1.1 (05 Dec 2025)**: Config file support, 3D visualisation, adaptive gap detection, consistent naming
- **v1.0 (03 Dec 2025)**: Complete rewrite with pymatgen for proper crystallographic symmetry
- **v0.4**: Attempted supercell approach
- **v0.3**: ICSD CIF compatibility (failed - low CN)
- **v0.2**: Mixed occupancy support
- **v0.1**: Original GSAS-II only version


import sys
print(f"Current Python: {sys.executable}")
print(f"Installing py3Dmol to this environment...\n")

!{sys.executable} -m pip install py3Dmol

print(f"\n✓ Installation complete - restart kernel to use py3Dmol")

# Once run, go to Kernel → Restart Kernel (or Restart & Clear Output)
# Then change this cell from Code to Markdown to prevent wasting time reinstalling the library every time.

In [107]:
# CELL 3: HELPER FUNCTIONS (EIR v1.2)
# ============================================================================
# Run this cell BEFORE Cell 4 (the main Eir code)
# NEW IN v1.2: Temperature and pressure extraction from CIF files

import os
import json

# ============================================================================
# ANION DETECTION
# ============================================================================
# Known anions that can form ionic bonds with metals
KNOWN_ANIONS = {
    # Chalcogens (Group 16)
    'O': -2, 'S': -2, 'Se': -2, 'Te': -2,
    # Halogens (Group 17)
    'F': -1, 'Cl': -1, 'Br': -1, 'I': -1,
    # Other common anions
    'N': -3, 'P': -3, 'As': -3, 'H': -1
}

def is_anion(element):
    """
    Check if an element is a known anion.
    
    Args:
        element: Element symbol (e.g., 'Cl', 'O', 'S')
    
    Returns:
        bool: True if element is in KNOWN_ANIONS
    """
    return element in KNOWN_ANIONS

def get_anion_charge(element):
    """
    Get typical charge for an anion.
    
    Args:
        element: Element symbol
    
    Returns:
        int: Typical charge (e.g., -1 for Cl, -2 for O)
    """
    return KNOWN_ANIONS.get(element, 0)

# ============================================================================
# CONFIG FILE SUPPORT
# ============================================================================

def load_oxidation_states_from_config(config_path=None, config_dict=None):
    """
    Load oxidation states from config file or dictionary.
    
    Args:
        config_path: Path to config.json file (optional)
        config_dict: Python dictionary with {element: oxidation_state} (optional)
    
    Returns:
        dict: {element: oxidation_state} or None if not found/invalid
    
    Example config.json:
    {
      "Na": 1,
      "Ni": 2,
      "Mn": 4,
      "O": -2,
      "Cl": -1
    }
    """
    # Priority 1: Use provided dictionary
    if config_dict is not None:
        if not isinstance(config_dict, dict):
            print("✗ Error: config_dict must be a dictionary")
            return None
        
        # Validate all values are integers
        try:
            validated = {str(k): int(v) for k, v in config_dict.items()}
            print(f"✓ Loaded oxidation states from provided dictionary:")
            for elem in sorted(validated.keys()):
                print(f"    {elem}: {validated[elem]:+d}")
            return validated
        except (ValueError, TypeError) as e:
            print(f"✗ Error validating config_dict: {e}")
            return None
    
    # Priority 2: Load from specified file
    if config_path is not None:
        if not os.path.exists(config_path):
            print(f"✗ Config file not found: {config_path}")
            return None
        
        try:
            with open(config_path, 'r') as f:
                data = json.load(f)
            
            # Validate format
            validated = {str(k): int(v) for k, v in data.items()}
            print(f"✓ Loaded oxidation states from {config_path}:")
            for elem in sorted(validated.keys()):
                print(f"    {elem}: {validated[elem]:+d}")
            return validated
        except json.JSONDecodeError as e:
            print(f"✗ Error parsing JSON in {config_path}: {e}")
            return None
        except (ValueError, TypeError) as e:
            print(f"✗ Error validating oxidation states in {config_path}: {e}")
            return None
    
    return None

def auto_detect_config_file(directory):
    """
    Automatically search for config.json in the target directory.
    
    Args:
        directory: Path to directory containing CIF files
    
    Returns:
        Path to config.json if found, None otherwise
    """
    config_path = os.path.join(directory, "config.json")
    if os.path.exists(config_path):
        print(f"✓ Found config.json in {directory}")
        return config_path
    return None

# ============================================================================
# 3D VISUALISATION OF PROBLEMATIC SITES (FIXED)
# ============================================================================

def visualize_problematic_site_3d(cif_path, element, site_label, bonds_info, 
                                   deviation, formal_charge, bvs_value, filename=None):
    """
    Create interactive 3D visualisation showing unit cell with highlighted problematic element.
    
    Args:
        cif_path: Path to CIF file
        element: Central atom element to highlight
        site_label: Crystallographic site label
        bonds_info: List of dicts with 'distance' and 'count' keys
        deviation: BVS deviation from formal charge
        formal_charge: Expected oxidation state
        bvs_value: Calculated BVS value
    
    Displays:
        Interactive 3D model of unit cell with color-coded atoms
    """
    try:
        import py3Dmol
        from IPython.display import display
    except ImportError:
        print("  [3D visualisation unavailable - py3Dmol not installed]")
        return
    
    try:
        # Create py3Dmol viewer
        view = py3Dmol.view(width=650, height=500)
        
        # Load CIF and replicate to show coordination better
        with open(cif_path, 'r', encoding='utf-8', errors='ignore') as f:
            cif_string = f.read()
        
        # Add 2x2x2 supercell for better visualisation
        view.addModel(cif_string, 'cif')
        view.replicateUnitCell(2, 2, 2)
        
        # Base style
        view.setStyle({'stick': {'radius': 0.2}, 'sphere': {'scale': 0.35}})
        
        # Highlight problematic element in BRIGHT RED (larger)
        view.addStyle({'elem': element}, 
                     {'sphere': {'color': 'red', 'scale': 0.7}})
        
        # Color anions differently based on type
        view.addStyle({'elem': 'O'}, {'sphere': {'color': 'blue', 'scale': 0.45}})
        view.addStyle({'elem': 'F'}, {'sphere': {'color': 'lightblue', 'scale': 0.40}})
        view.addStyle({'elem': 'Cl'}, {'sphere': {'color': 'green', 'scale': 0.50}})
        view.addStyle({'elem': 'Br'}, {'sphere': {'color': 'brown', 'scale': 0.55}})
        view.addStyle({'elem': 'I'}, {'sphere': {'color': 'purple', 'scale': 0.60}})
        view.addStyle({'elem': 'S'}, {'sphere': {'color': 'yellow', 'scale': 0.50}})
        view.addStyle({'elem': 'Se'}, {'sphere': {'color': 'orange', 'scale': 0.52}})
        view.addStyle({'elem': 'Te'}, {'sphere': {'color': 'gold', 'scale': 0.55}})
        
        # Other metals in GREY
        other_metals = ['Na', 'K', 'Li', 'Cr', 'Ti', 'Mn', 'Ni', 'Co', 'Fe', 'V', 'Sb', 'Mg', 
                       'Ca', 'Sr', 'Ba', 'Zn', 'Cd', 'Cu', 'Al']
        for metal in other_metals:
            if metal != element:
                view.addStyle({'elem': metal}, 
                             {'sphere': {'color': 'grey', 'scale': 0.4}})
        
        view.zoomTo()
        view.rotate(45, {'x': 1, 'y': 1, 'z': 0})  # Better viewing angle
        
        # Extract CollCode from filename if present
        file_id = ""
        if filename:
            # Try to extract CollCode (e.g., "CollCode253189" from filename)
            import re
            collcode_match = re.search(r'CollCode(\d+)', filename)
            if collcode_match:
                file_id = f" in CollCode{collcode_match.group(1)}"
            else:
                # Just use filename without extension
                file_id = f" ({filename.replace('.cif', '')})"
        
        # Print info box
        print(f"\n  ╔══════════════════════════════════════════════════════════════╗")
        print(f"  ║ 3D VISUALISATION: {element} at site {site_label}{file_id}")
        print(f"  ╠══════════════════════════════════════════════════════════════╣")
        print(f"  ║ BVS: {bvs_value:.3f}  │  Expected: {formal_charge:+d}  │  Δ: {deviation:+.3f} v.u.")
        print(f"  ╠══════════════════════════════════════════════════════════════╣")
        print(f"  ║ Coordination Environment:")
        
        if bonds_info:
            for bond in bonds_info[:6]:
                print(f"  ║   • {bond['count']}× {bond['distance']:.4f} Å")
        else:
            print(f"  ║   • CN information not available")
            
        print(f"  ╠══════════════════════════════════════════════════════════════╣")
        print(f"  ║ Shows: 2×2×2 supercell (8 unit cells)")
        print(f"  ║ RED = {element} (problematic) │ Colored = anions │ GREY = other metals")
        print(f"  ╚══════════════════════════════════════════════════════════════╝")
        
        # Display the interactive view
        display(view)
        print()
        
    except Exception as e:
        print(f"  [3D visualisation failed: {e}]")
        import traceback
        traceback.print_exc()

# ============================================================================
# JAHN-TELLER ION DETECTION
# ============================================================================
# Jahn-Teller active ions (d4 high-spin, d9, low-spin d7)
JAHN_TELLER_IONS = {
    'Cr': [2],      # Cr2+: d4 high-spin
    'Mn': [3],      # Mn3+: d4 high-spin
    'Cu': [2],      # Cu2+: d9
    'Ni': [3],      # Ni3+: d7 low-spin (less common)
    'Co': [2]       # Co2+: d7 high-spin (weak JT)
}

def is_jahn_teller_ion(element, valence):
    """
    Check if an element/valence combination is Jahn-Teller active.
    
    Args:
        element: Element symbol (e.g., 'Cu')
        valence: Oxidation state (e.g., 2)
    
    Returns:
        bool: True if Jahn-Teller active
    """
    return element in JAHN_TELLER_IONS and abs(valence) in JAHN_TELLER_IONS[element]

# ============================================================================
# ADAPTIVE GAP DETECTION FOR COORDINATION
# ============================================================================

def detect_coordination_adaptive(bond_distances, element=None, valence=None, min_gap=0.05):
    """
    Adaptive coordination number detection with special handling for Jahn-Teller ions.
    
    For Jahn-Teller active ions (e.g., Cu2+, Mn3+), uses relaxed gap threshold
    to avoid over-splitting the coordination sphere.
    
    Args:
        bond_distances: Sorted list of bond distances (Å)
        element: Element symbol (optional, for JT detection)
        valence: Oxidation state (optional, for JT detection)
        min_gap: Minimum gap size to consider (Å), default 0.05
    
    Returns:
        int: Coordination number (bonds before largest gap)
    """
    if len(bond_distances) < 2:
        return len(bond_distances)
    
    # Check if this is a Jahn-Teller ion
    is_jt = False
    if element and valence:
        is_jt = is_jahn_teller_ion(element, valence)
    
    # For JT ions, use larger minimum gap threshold
    effective_min_gap = min_gap * 2.0 if is_jt else min_gap
    
    # Calculate gaps between consecutive bonds
    gaps = []
    for i in range(len(bond_distances) - 1):
        gap = bond_distances[i+1] - bond_distances[i]
        if gap >= effective_min_gap:  # Only consider significant gaps
            gaps.append((gap, i+1))  # Store (gap_size, position_after_gap)
    
    if not gaps:
        # No significant gaps found - return all bonds
        return len(bond_distances)
    
    # Find largest gap
    largest_gap, cn = max(gaps, key=lambda x: x[0])
    
    return cn

# ============================================================================
# EIR v1.2: ICSD METADATA EXTRACTION WITH TEMPERATURE & PRESSURE
# ============================================================================

def extract_icsd_metadata(cif_path):
    """
    Extract publication metadata AND measurement conditions from ICSD CIF file.
    
    v1.2:
    - Extracts measurement temperature from _diffrn_ambient_temperature
    - Extracts measurement pressure from _diffrn_ambient_pressure (in kPa)
    - Classifies measurement conditions (ambient/high_T/high_P/unknown)
    
    Args:
        cif_path: Path to CIF file
    
    Returns:
        dict with keys: 
        - Original: year, journal, authors, r_factor, icsd_code, creation_date, density
        - NEW: measurement_temperature_K, measurement_pressure_Pa, measurement_pressure_GPa, 
               measurement_conditions
    """
    metadata = {
        # Original fields
        'icsd_code': None,
        'creation_date': None,
        'year': None,
        'journal': None,
        'authors': [],
        'r_factor': None,
        'density': None,
        
        # Measurement condition fields
        'measurement_temperature_K': None,
        'measurement_pressure_Pa': None,
        'measurement_pressure_GPa': None,
        'measurement_conditions': 'unknown'
    }
    
    try:
        with open(cif_path, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
        
        in_citation_loop = False
        in_author_loop = False
        citation_columns = []
        author_columns = []
        
        for i, line in enumerate(lines):
            line_stripped = line.strip()
            
            # ICSD code
            if line_stripped.startswith('_database_code_ICSD'):
                parts = line_stripped.split()
                if len(parts) > 1:
                    metadata['icsd_code'] = parts[1]
            
            # Creation date
            elif line_stripped.startswith('_audit_creation_date'):
                parts = line_stripped.split()
                if len(parts) > 1:
                    metadata['creation_date'] = parts[1]
            
            # Density
            elif line_stripped.startswith('_exptl_crystal_density_diffrn'):
                parts = line_stripped.split()
                if len(parts) > 1:
                    try:
                        metadata['density'] = float(parts[1])
                    except ValueError:
                        pass
            
            # ========================================================================
            # MEASUREMENT TEMPERATURE
            # ========================================================================
            elif line_stripped.startswith('_diffrn_ambient_temperature'):
                parts = line_stripped.split()
                if len(parts) > 1:
                    try:
                        # Remove any parentheses with errors: "298(2)" -> 298
                        temp_str = parts[1].split('(')[0].strip()
                        metadata['measurement_temperature_K'] = float(temp_str)
                    except ValueError:
                        pass
            
            # Alternative temperature field (some CIFs use this)
            elif line_stripped.startswith('_cell_measurement_temperature'):
                parts = line_stripped.split()
                if len(parts) > 1 and metadata['measurement_temperature_K'] is None:
                    try:
                        temp_str = parts[1].split('(')[0].strip()
                        metadata['measurement_temperature_K'] = float(temp_str)
                    except ValueError:
                        pass
            
            # ========================================================================
            # MEASUREMENT PRESSURE
            # ========================================================================
            elif line_stripped.startswith('_diffrn_ambient_pressure'):
                parts = line_stripped.split()
                if len(parts) > 1:
                    try:
                        # Remove any parentheses with errors
                        pressure_str = parts[1].split('(')[0].strip()
                        # CRITICAL: CIF standard uses kilopascals (kPa), not Pa!
                        pressure_kpa = float(pressure_str)
                        metadata['measurement_pressure_Pa'] = pressure_kpa * 1000  # Convert kPa → Pa
                        
                        # Convert to GPa for convenience (1 GPa = 1e6 kPa)
                        metadata['measurement_pressure_GPa'] = pressure_kpa / 1e6
                    except ValueError:
                        pass
            
            # Alternative pressure field
            elif line_stripped.startswith('_cell_measurement_pressure'):
                parts = line_stripped.split()
                if len(parts) > 1 and metadata['measurement_pressure_Pa'] is None:
                    try:
                        pressure_str = parts[1].split('(')[0].strip()
                        pressure_kpa = float(pressure_str)
                        metadata['measurement_pressure_Pa'] = pressure_kpa * 1000
                        metadata['measurement_pressure_GPa'] = pressure_kpa / 1e6
                    except ValueError:
                        pass
            
            # R-factor (multiple possible tags)
            elif any(tag in line_stripped for tag in ['_refine_ls_R_factor', '_refine_ls_wR_factor']):
                parts = line_stripped.split()
                if len(parts) > 1:
                    try:
                        r_val = float(parts[1])
                        if metadata['r_factor'] is None or r_val < metadata['r_factor']:
                            metadata['r_factor'] = r_val
                    except ValueError:
                        pass
            
            # Citation loop - collect column headers
            elif line_stripped.startswith('loop_'):
                in_citation_loop = False
                in_author_loop = False
                citation_columns = []
                author_columns = []
            
            elif line_stripped.startswith('_citation_') and not line_stripped.startswith('_citation_author'):
                in_citation_loop = True
                citation_columns.append(line_stripped)
            
            elif line_stripped.startswith('_citation_author'):
                in_author_loop = True
                in_citation_loop = False
                author_columns.append(line_stripped)
            
            # Parse citation data line
            elif in_citation_loop and citation_columns and not line_stripped.startswith('_') and not line_stripped.startswith('#') and line_stripped:
                import shlex
                try:
                    values = shlex.split(line_stripped)
                    
                    # Map columns to values
                    for col_idx, col_name in enumerate(citation_columns):
                        if col_idx < len(values):
                            value = values[col_idx]
                            
                            if '_citation_year' in col_name:
                                try:
                                    metadata['year'] = int(value)
                                except ValueError:
                                    pass
                            
                            elif '_citation_journal_full' in col_name:
                                metadata['journal'] = value
                
                except ValueError:
                    pass
                
                in_citation_loop = False
            
            # Parse author data lines
            elif in_author_loop and author_columns and not line_stripped.startswith('_') and not line_stripped.startswith('#') and not line_stripped.startswith('loop_') and line_stripped:
                import shlex
                try:
                    values = shlex.split(line_stripped)
                    
                    for col_idx, col_name in enumerate(author_columns):
                        if '_citation_author_name' in col_name and col_idx < len(values):
                            author = values[col_idx]
                            if author and author not in metadata['authors']:
                                metadata['authors'].append(author)
                
                except ValueError:
                    pass
            
            # End author loop on new loop or non-data line
            elif in_author_loop and (line_stripped.startswith('_') or line_stripped.startswith('loop_')):
                in_author_loop = False
        
        # Limit authors to first 3 + "et al" if more
        if len(metadata['authors']) > 3:
            metadata['authors'] = metadata['authors'][:3] + ['et al.']
        
        # ========================================================================
        # CLASSIFY MEASUREMENT CONDITIONS
        # ========================================================================
        temp_K = metadata['measurement_temperature_K']
        pressure_GPa = metadata['measurement_pressure_GPa']
        
        # Define thresholds
        TEMP_AMBIENT_MAX = 350  # K (77°C) - room temperature range
        TEMP_HIGH = 500  # K (227°C) - clearly elevated
        
        PRESSURE_AMBIENT_MAX = 0.0002  # GPa (0.2 MPa = 2 bar) - near atmospheric
        PRESSURE_HIGH = 0.5  # GPa (5 kbar) - clearly elevated
        
        if temp_K is not None or pressure_GPa is not None:
            # Start with known condition
            is_high_temp = False
            is_high_pressure = False
            is_ambient_temp = True  # Assume ambient unless proven otherwise
            is_ambient_pressure = True
            
            # Check temperature
            if temp_K is not None:
                if temp_K > TEMP_HIGH:
                    is_high_temp = True
                    is_ambient_temp = False
                elif temp_K > TEMP_AMBIENT_MAX:
                    # Elevated but not clearly "high" - still not ambient
                    is_ambient_temp = False
            
            # Check pressure
            if pressure_GPa is not None:
                if pressure_GPa > PRESSURE_HIGH:
                    is_high_pressure = True
                    is_ambient_pressure = False
                elif pressure_GPa > PRESSURE_AMBIENT_MAX:
                    # Elevated but not clearly "high" - still not ambient
                    is_ambient_pressure = False
            
            # Classify
            if is_high_temp and is_high_pressure:
                metadata['measurement_conditions'] = 'high_T_high_P'
            elif is_high_temp:
                metadata['measurement_conditions'] = 'high_T'
            elif is_high_pressure:
                metadata['measurement_conditions'] = 'high_P'
            elif is_ambient_temp and is_ambient_pressure:
                metadata['measurement_conditions'] = 'ambient'
            else:
                # Elevated but not clearly high
                metadata['measurement_conditions'] = 'elevated'
        else:
            # No temperature or pressure data
            metadata['measurement_conditions'] = 'unknown'
        
    except Exception as e:
        print(f"  [Warning: Could not extract all metadata: {e}]")
    
    return metadata

# ============================================================================
# GEOMETRY VALIDATION
# ============================================================================

def check_geometry_validity(supercell_neighbors, site_info, element_categories):
    """
    Check O-O and M-M distances for geometric validity.
    
    Args:
        supercell_neighbors: List of neighbor data from gemmi supercell
        site_info: Dict with 'element', 'label', 'coords'
        element_categories: Dict mapping elements to categories
    
    Returns:
        dict with validation results and warnings
    """
    results = {
        'o_o_distances': [],
        'o_o_min': None,
        'o_o_warnings': [],
        'm_m_distances': [],
        'm_m_min': None,
        'm_m_warnings': [],
        'geometry_flags': []
    }
    
    try:
        central_element = site_info['element']
        central_category = element_categories.get(central_element, {}).get('category', 'unknown')
        
        # Extract O-O and M-M distances from neighbors
        for neighbor in supercell_neighbors:
            neighbor_elem = neighbor.get('element', '')
            neighbor_cat = element_categories.get(neighbor_elem, {}).get('category', 'unknown')
            distance = neighbor.get('distance', 0)
            
            # O-O distances (only if central atom is oxygen)
            if central_element == 'O' and neighbor_elem == 'O':
                results['o_o_distances'].append(distance)
            
            # M-M distances (metal-metal)
            elif (central_category in ['alkali_metals', 'alkaline_earth_metals', 'transition_metals_3d', 
                                       'transition_metals_4d', 'transition_metals_5d', 'lanthanides'] and
                  neighbor_cat in ['alkali_metals', 'alkaline_earth_metals', 'transition_metals_3d',
                                   'transition_metals_4d', 'transition_metals_5d', 'lanthanides']):
                results['m_m_distances'].append(distance)
        
        # Analyze O-O distances
        if results['o_o_distances']:
            results['o_o_min'] = min(results['o_o_distances'])
            
            # Expected O-O edge length in octahedra: ~2.7-2.9 Å
            # Face-sharing can be as low as ~2.3 Å
            if results['o_o_min'] < 2.3:
                results['o_o_warnings'].append(f"O-O distance {results['o_o_min']:.3f} Å unusually short (< 2.3 Å)")
                results['geometry_flags'].append('SHORT_O-O')
            elif results['o_o_min'] < 2.5:
                results['o_o_warnings'].append(f"O-O distance {results['o_o_min']:.3f} Å short (face-sharing?)")
        
        # Analyze M-M distances  
        if results['m_m_distances']:
            results['m_m_min'] = min(results['m_m_distances'])
            
            # Metal-metal distances should generally be > 2.5 Å for edge-sharing
            # Direct M-M bonding is < 3.0 Å but rare in oxides
            if results['m_m_min'] < 2.5:
                results['m_m_warnings'].append(f"M-M distance {results['m_m_min']:.3f} Å unusually short")
                results['geometry_flags'].append('SHORT_M-M')
        
    except Exception as e:
        results['geometry_flags'].append('VALIDATION_ERROR')
        print(f"  [Geometry validation error: {e}]")
    
    return results

# ============================================================================
# MIXED OCCUPANCY DETECTION & ANALYSIS
# ============================================================================

def extract_site_occupancy_info(data_block, site_label):
    """
    Extract occupancy information for a specific crystallographic site.
    
    Args:
        data_block: PyCifRW data block
        site_label: Site label to look up (e.g., "Li1", "Nb1")
    
    Returns:
        dict with occupancy info or None if not found
    """
    try:
        labels = data_block.get('_atom_site_label', [])
        types = data_block.get('_atom_site_type_symbol', [])
        occupancies = data_block.get('_atom_site_occupancy', [])
        
        if not labels or not types:
            return None
        
        # Find all atoms at this site position
        site_data = []
        for i, label in enumerate(labels):
            if label.strip() == site_label:
                element = types[i].strip().rstrip('+-0123456789')
                element = element.capitalize() if len(element) > 1 else element.upper()
                
                # Get occupancy (default 1.0 if not specified)
                occ = 1.0
                if occupancies and i < len(occupancies):
                    try:
                        occ_str = occupancies[i].strip()
                        # Handle errors in parentheses: "0.75(2)" -> 0.75
                        occ = float(occ_str.split('(')[0])
                    except (ValueError, AttributeError):
                        occ = 1.0
                
                site_data.append({
                    'element': element,
                    'occupancy': occ
                })
        
        return site_data if site_data else None
    
    except Exception as e:
        return None

def analyze_mixed_occupancy_at_site(site_coords, site_data, final_corrected_data, block_name):
    """
    Analyze mixed occupancy at a crystallographic site.
    
    Returns dict with:
        - has_mixed_occupancy: bool
        - occupancy_string: str (e.g., "Li:0.75,Nb:0.25")
        - n_elements_at_site: int
        - disorder_type: 'pure' / 'homovalent' / 'heterovalent'
        - valence_difference: float
        - max/min_valence_at_site: int
    """
    result = {
        'has_mixed_occupancy': False,
        'occupancy_string': 'unknown',
        'n_elements_at_site': 1,
        'disorder_type': 'pure',
        'valence_difference': 0.0,
        'max_valence_at_site': None,
        'min_valence_at_site': None
    }
    
    elements = site_data.get('elements', [])
    if not elements:
        return result
    
    # Get valences from final_corrected_data
    valences = []
    
    # Find the matching site in final_corrected_data to get confirmed_valences
    if block_name in final_corrected_data:
        for coords, site_info in final_corrected_data[block_name]['sites_with_bonds'].items():
            if coords == site_coords:  # Match by coordinates
                confirmed_valences = site_info.get('confirmed_valences', {})
                for elem in elements:
                    val = confirmed_valences.get(elem)
                    if val is not None:
                        valences.append(abs(val))
                break
    
    # If we couldn't find valences from final_corrected_data, try from element_data
    if not valences:
        for elem in elements:
            elem_data = site_data.get('element_data', {}).get(elem, {})
            val = elem_data.get('valence')
            if val is not None:
                valences.append(abs(val))
    
    result['n_elements_at_site'] = len(elements)
    
    if valences:
        result['max_valence_at_site'] = int(max(valences))
        result['min_valence_at_site'] = int(min(valences))
        result['valence_difference'] = float(max(valences) - min(valences))
    
    # Build occupancy string
    if len(elements) > 1:
        result['has_mixed_occupancy'] = True
        # Assume equal distribution for now (proper implementation would extract from CIF)
        occ_parts = []
        for elem in elements:
            occ = 1.0 / len(elements)
            occ_parts.append(f"{elem}:{occ:.2f}")
        result['occupancy_string'] = ','.join(occ_parts)
    else:
        result['occupancy_string'] = f"{elements[0]}:1.00"
    
    # Classify disorder type
    if len(elements) == 1:
        result['disorder_type'] = 'pure'
        result['valence_difference'] = 0.0
    elif not valences:
        result['disorder_type'] = 'heterovalent'
        result['valence_difference'] = 0.0
    else:
        # Check if all valences are the same
        if len(set(valences)) == 1:
            result['disorder_type'] = 'homovalent'
            result['valence_difference'] = 0.0
        else:
            result['disorder_type'] = 'heterovalent'
            result['valence_difference'] = float(max(valences) - min(valences))
    
    return result

def calculate_discrepancy_factor(bvs_value, formal_valence):
    """
    Calculate d1 = (S_i - V_i) from Bosi (2014).
    
    Positive d1: overbonded (compressed bonds)
    Negative d1: underbonded (stretched bonds)
    """
    if bvs_value is None or formal_valence is None:
        return None, None, 'unknown'
    
    d1 = bvs_value - abs(formal_valence)
    abs_d1 = abs(d1)
    
    if d1 > 0.05:
        bonding_state = 'overbonded'
    elif d1 < -0.05:
        bonding_state = 'underbonded'
    else:
        bonding_state = 'balanced'
    
    return d1, abs_d1, bonding_state

def assign_confidence_flag(disorder_type, valence_diff, abs_d1, deviation):
    """
    Assign confidence level for BVS result based on disorder and strain.
    """
    abs_dev = abs(deviation) if deviation is not None else 0
    abs_d1_val = abs_d1 if abs_d1 is not None else 0
    
    # Initialize flags
    requires_review = False
    bosi_artifact_likely = False
    
    # Check for spectroscopy requirement (Bosi 2014: heterovalent ΔV≥3)
    if disorder_type == 'heterovalent' and valence_diff >= 3:
        confidence = 'SPECTROSCOPY_REQUIRED'
        bosi_artifact_likely = True
        requires_review = True
        return confidence, bosi_artifact_likely, requires_review
    
    # Check for manual review needs
    if abs_dev > 0.30 or abs_d1_val > 0.20:
        requires_review = True
    
    # Assign confidence based on disorder type and strain
    if disorder_type == 'pure':
        if abs_d1_val < 0.10:
            confidence = 'HIGH'
        elif abs_d1_val < 0.20:
            confidence = 'MEDIUM'
        else:
            confidence = 'LOW'
            requires_review = True
    
    elif disorder_type == 'homovalent':
        if abs_d1_val < 0.15:
            confidence = 'MEDIUM'
        elif abs_d1_val < 0.25:
            confidence = 'LOW'
        else:
            confidence = 'LOW'
            requires_review = True
    
    elif disorder_type == 'heterovalent':
        if valence_diff < 2:
            confidence = 'MEDIUM'
        elif valence_diff < 3:
            confidence = 'LOW'
            requires_review = True
        else:
            # ΔV ≥ 3: Bosi artifact likely
            confidence = 'SPECTROSCOPY_REQUIRED'
            bosi_artifact_likely = True
            requires_review = True
    
    else:
        confidence = 'LOW'
        requires_review = True
    
    return confidence, bosi_artifact_likely, requires_review

# ============================================================================
# INITIALISATION MESSAGE
# ============================================================================

print("✓ Eir v1.2 helper functions loaded successfully")
print("  • Multi-anion support enabled (O, F, Cl, Br, I, S, Se, Te, etc.)")
print("  • Config file support enabled")
print("  • 3D visualisation enabled (if py3Dmol installed)")
print("  • Adaptive gap detection enabled")
print("  • Temperature/Pressure extraction enabled (v1.2)")
print("  • ICSD metadata extraction with conditions classification")
print("  • Geometry validation enabled")
print("  • Mixed occupancy analysis enabled")

✓ Eir v1.2 helper functions loaded successfully
  • Multi-anion support enabled (O, F, Cl, Br, I, S, Se, Te, etc.)
  • Config file support enabled
  • 3D visualisation enabled (if py3Dmol installed)
  • Adaptive gap detection enabled
  • Temperature/Pressure extraction enabled (v1.2)
  • ICSD metadata extraction with conditions classification
  • Geometry validation enabled
  • Mixed occupancy analysis enabled


In [108]:
## CELL 4: MAIN EIR FUNCTIONALITY
# ============================================================================
# Run this cell BEFORE Cell 5

"""
Eir Batch Processor v1.2 - Automated Bond Valence Sum Analysis
================================================================

NEW IN v1.2:
- Multi-anion support: Handles O, F, Cl, Br, I, S, Se, Te, etc.
- Auto-detects M-X bonds (not just M-O)
- Merged BVS parameters (Gagné & Hawthorne 2015 + Brown 2020)
- 1,385 M-X parameter pairs vs 168 M-O pairs (8× expansion)
- Temperature extraction from CIF files (_diffrn_ambient_temperature)
- Pressure extraction from CIF files (_diffrn_ambient_pressure)
- Automatic classification of measurement conditions
- 4 new columns in CSV output for condition tracking


NEW IN v1.1:
- Config file support (config.json) for unattended batch processing
- Automatic 3D visualisation of problematic sites (|deviation| > 0.3 v.u.)
- Adaptive gap detection for Jahn-Teller active ions
- Enhanced output with deviation column

FEATURES:
- ICSD & GSAS-II CIF support with proper crystallographic symmetry
- Supercell generation for high-symmetry space groups (R-3m, P63/mmc)
- Mixed/partial occupancy support (e.g., Na₂/₃, Cr₀.₈Ti₀.₂)
- Gagné & Hawthorne (2015) + Brown (2020) bond valence parameters
- Global Instability Index (GII) calculation
- Batch processing with detailed CSV output

OUTPUT:
All results saved to ./outputs/ subfolder:
- eir_batch_results_v1.2_YYYYMMDD_HHMMSS.csv
- oxidation_states_used_YYYYMMDD_HHMMSS.json
- eir_batch_failed_v1.2_YYYYMMDD_HHMMSS.csv (if any failures)

CITATIONS:
Gagné & Hawthorne (2015) Acta Cryst. B71, 561-578
Brown (2020) IUCr Bond Valence Parameters
Brown & Altermatt (1985) Acta Cryst. B41, 244-247
Brese & O'Keeffe (1991) Acta Cryst. B47, 192-197
"""

import os
import sys
import json
import re
import math
from collections import Counter, defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# CIF parsing
try:
    from CifFile import ReadCif
except ImportError:
    print("ERROR: PyCifRW not found. Install with: pip install pycifrw")
    sys.exit(1)

# pymatgen for ICSD CIF bond calculation (robust parser + proper crystallography)
try:
    from pymatgen.core import Structure
    from pymatgen.io.cif import CifParser
    PYMATGEN_AVAILABLE = True
    print("✓ pymatgen available - ICSD CIF files supported with proper crystallographic symmetry")
except ImportError:
    PYMATGEN_AVAILABLE = False
    print("⚠ pymatgen not available - ICSD CIFs will fail. Install with: pip install pymatgen")
    print("  pymatgen is REQUIRED for proper handling of:")
    print("   - Partial occupancy (e.g., Na2/3 sites)")
    print("   - Mixed occupancy (e.g., Cr0.8Ti0.2)")
    print("   - All space groups")
    print("   - Robust CIF parsing (handles ICSD formatting quirks)")

# py3Dmol for 3D visualisation
try:
    import py3Dmol
    from IPython.display import display
    PY3DMOL_AVAILABLE = True
    print("✓ py3Dmol available - 3D visualisation enabled for problematic sites")
except ImportError as e:
    PY3DMOL_AVAILABLE = False
    print("⚠ py3Dmol not available - Install with: pip install py3Dmol")
    print(f"   Import error: {e}")
except Exception as e:
    PY3DMOL_AVAILABLE = False
    print("⚠ py3Dmol import failed - Unexpected error")
    print(f"   Error: {e}")

# ============================================================================
# DATABASE MANAGER (From Skuld 1.5 - UNCHANGED)
# ============================================================================

class DatabaseManager:
    """Manages external JSON databases for ionic radii and bond valence parameters"""
    
    def __init__(self, data_dir="./Data"):
        self.data_dir = data_dir
        self.shannon_radii = None
        self.bond_valence_params = None
        self.oxidation_states = None
        self.element_categories = None
        
        if not os.path.exists(self.data_dir):
            print(f"Warning: Data directory '{self.data_dir}' not found!")
            print("Please ensure your JSON database files are in the ./Data folder")
    
    def load_shannon_radii(self):
        """Load Shannon-Prewitt ionic radii from JSON file"""
        if self.shannon_radii is not None:
            return self.shannon_radii
            
        filepath = os.path.join(self.data_dir, "shannon_radii.json")
        
        if not os.path.exists(filepath):
            print(f"Shannon radii file not found at {filepath}")
            return None
        
        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
                self.shannon_radii = {}
                for element, valence_dict in data.items():
                    for valence_str, cn_dict in valence_dict.items():
                        valence = int(valence_str)
                        for cn_str, radius in cn_dict.items():
                            cn = int(cn_str)
                            self.shannon_radii[(element, valence, cn)] = radius
                
            print(f"✓ Loaded Shannon radii database with {len(self.shannon_radii)} entries")
        except Exception as e:
            print(f"✗ Error loading Shannon radii: {e}")
            return None
        
        return self.shannon_radii
    
    def load_bond_valence_params(self):
        """Load Gagné & Hawthorne bond valence parameters from JSON file"""
        if self.bond_valence_params is not None:
            return self.bond_valence_params
            
        filepath = os.path.join(self.data_dir, "bvs_parameters.json")
        
        if not os.path.exists(filepath):
            print(f"BVS parameters file not found at {filepath}")
            return None
        
        try:
            import ast
            with open(filepath, 'r') as f:
                data = json.load(f)
                self.bond_valence_params = {}
                for key, params in data.items():
                    # Keys are string representations of tuples: "('Na', 1, 'O')"
                    try:
                        cation, cation_valence, anion = ast.literal_eval(key)
                        self.bond_valence_params[(cation, cation_valence, anion)] = {
                            'R0': params['r0'],  # Note lowercase in JSON
                            'B': params.get('b', 0.37)  # Note lowercase in JSON
                        }
                    except (ValueError, SyntaxError) as e:
                        # Skip malformed keys
                        continue
            
            print(f"✓ Loaded BVS parameters database with {len(self.bond_valence_params)} entries")
        except Exception as e:
            print(f"✗ Error loading BVS parameters: {e}")
            import traceback
            traceback.print_exc()
            return None
        
        return self.bond_valence_params
    
    def load_common_oxidation_states(self):
        """Load common oxidation states from JSON file"""
        if self.oxidation_states is not None:
            return self.oxidation_states
            
        filepath = os.path.join(self.data_dir, "common_oxidation_states.json")
        
        if not os.path.exists(filepath):
            print(f"Oxidation states file not found at {filepath}")
            return None
        
        try:
            with open(filepath, 'r') as f:
                self.oxidation_states = json.load(f)
            
            print(f"✓ Loaded oxidation states database with {len(self.oxidation_states)} elements")
        except Exception as e:
            print(f"✗ Error loading oxidation states: {e}")
            return None
        
        return self.oxidation_states
    
    def load_element_categories(self):
        """Load element categories from JSON file"""
        if self.element_categories is not None:
            return self.element_categories
            
        filepath = os.path.join(self.data_dir, "element_categories.json")
        
        if not os.path.exists(filepath):
            print(f"Element categories file not found at {filepath}")
            return None
        
        try:
            with open(filepath, 'r') as f:
                self.element_categories = json.load(f)
            
            print(f"✓ Loaded element categories database")
        except Exception as e:
            print(f"✗ Error loading element categories: {e}")
            return None
        
        return self.element_categories
    
    def get_ionic_radius(self, element, valence, cn):
        """Get ionic radius for given element, valence, and coordination number"""
        if self.shannon_radii is None:
            self.load_shannon_radii()
        
        if self.shannon_radii is None:
            return None
        
        return self.shannon_radii.get((element, valence, cn))
    
    def get_bv_parameters(self, cation, cation_valence, anion):
        """Get R0 and B parameters for BVS calculation"""
        if self.bond_valence_params is None:
            self.load_bond_valence_params()
        
        if self.bond_valence_params is None:
            return None, None
        
        params = self.bond_valence_params.get((cation, cation_valence, anion))
        if params:
            return params['R0'], params['B']
        return None, None
    
    def get_common_oxidation_states(self, element):
        """Get list of common oxidation states for an element"""
        if self.oxidation_states is None:
            self.load_common_oxidation_states()
        
        if self.oxidation_states is None:
            return []
        
        return self.oxidation_states.get(element, [])
    
    def get_element_category(self, element):
        """Get category for an element (alkali, transition, etc.)"""
        if self.element_categories is None:
            self.load_element_categories()
        
        if self.element_categories is None:
            return "unknown"
        
        for category, elements in self.element_categories.items():
            if element in elements:
                return category
        return "other"

# Initialize global database manager
db_manager = DatabaseManager()
db_manager.load_shannon_radii()
db_manager.load_bond_valence_params()
db_manager.load_common_oxidation_states()
db_manager.load_element_categories()

# ============================================================================
# UTILITY FUNCTIONS (From Skuld 1.5 - UNCHANGED)
# ============================================================================

def parse_value_with_error(value_string):
    """
    Parse CIF value strings that may contain errors in parentheses.
    Example: "1.2345(67)" -> 1.2345, 0.0067
    """
    if not value_string or value_string == '.' or value_string == '?':
        return None, None
    
    match = re.match(r'^([+-]?\d+\.?\d*(?:[eE][+-]?\d+)?)\((\d+)\)$', value_string.strip())
    if match:
        value_str, error_str = match.groups()
        value = float(value_str)
        
        if '.' in value_str:
            decimal_places = len(value_str.split('.')[1].split('e')[0].split('E')[0])
        else:
            decimal_places = 0
        
        error = float(error_str) * (10 ** (-decimal_places))
        return value, error
    else:
        try:
            value = float(value_string.strip())
            return value, None
        except ValueError:
            return None, None

def validate_element_combination(cation, anion):
    """Check if cation-anion combination is chemically reasonable"""
    cation_category = db_manager.get_element_category(cation)
    anion_category = db_manager.get_element_category(anion)
    
    if cation_category == "other" and anion_category == "other":
        return False
    
    return True

# ============================================================================
# BOND DISTANCE CALCULATION FOR ICSD CIF FILES
# ============================================================================

def clean_cif_for_gemmi(cif_file_path):
    """
    Pre-process CIF file to remove lines that cause gemmi parsing errors.
    
    Removes _chemical_name_structure_type which often has UTF-8 encoding issues in ICSD files.
    This tag is not needed for bond calculation.
    
    Returns cleaned CIF as string for gemmi to parse.
    """
    try:
        # Read file with UTF-8 encoding, replacing errors
        with open(cif_file_path, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()
        
        cleaned_lines = []
        i = 0
        while i < len(lines):
            line = lines[i]
            
            # Check if this line contains the problematic tag
            if '_chemical_name_structure_type' in line:
                # Skip this line
                # Also skip the next line if it doesn't start with _ or # (it's the value)
                i += 1
                if i < len(lines):
                    next_line = lines[i].strip()
                    # If next line is a value (not a tag), skip it too
                    if next_line and not next_line.startswith('_') and not next_line.startswith('#') and not next_line.startswith('loop_') and not next_line.startswith('data_'):
                        i += 1
                continue
            
            cleaned_lines.append(line)
            i += 1
        
        cleaned = ''.join(cleaned_lines)
        # Debug info
        # print(f"  Debug: Removed {len(lines) - len(cleaned_lines)} lines")
        return cleaned
    
    except Exception as e:
        print(f"  Warning: Error cleaning CIF file: {e}")
        return None


def calculate_bonds_from_positions(cif_file_path, max_bond_distance=3.5):
    """
    Calculate bond distances from CIF using proper crystallographic symmetry operations.
    
    Uses gemmi library for robust handling of:
    - Partial occupancy (e.g., Na site 2/3 occupied)  
    - Mixed occupancy (e.g., Cr0.8Ti0.2 on same site)
    - All space groups (R-3m, P63/mmc, C2/m, etc.)
    - Proper symmetry code generation
    
    CRITICAL: ICSD files are small molecule/inorganic structures, not macromolecular.
    Uses SmallStructure, not Structure.
    
    Args:
        cif_file_path: Path to CIF file
        max_bond_distance: Maximum bond distance to consider (Ångströms)
    
    Returns:
        dict with bond data formatted like GSAS-II _geom_bond output, or None if failed
    """
    try:
        import gemmi
    except ImportError:
        print("  ✗ gemmi not available - install with: pip install gemmi")
        print("    gemmi is required for proper crystallographic bond calculation")
        return None
    
    import tempfile
    
    try:
        # Read file with UTF-8 error handling
        cleaned_cif = clean_cif_for_gemmi(cif_file_path)
        if cleaned_cif is None:
            print("  ✗ Could not read CIF file")
            return None
        
        # Write to temporary file
        with tempfile.NamedTemporaryFile(mode='w', suffix='.cif', delete=False, encoding='utf-8') as tmp:
            tmp.write(cleaned_cif)
            tmp_path = tmp.name
        
        try:
            # Read CIF document
            doc = gemmi.cif.read(tmp_path)
            block = doc.sole_block()
            
            # CRITICAL: Use SmallStructure for ICSD (small molecule/inorganic) CIFs
            # NOT Structure (macromolecular)
            small_struct = gemmi.make_small_structure_from_block(block)
            
        finally:
            # Clean up temp file
            import os
            try:
                os.unlink(tmp_path)
            except:
                pass
        
        if not small_struct or len(small_struct.sites) == 0:
            print("  ✗ No atomic sites found in CIF")
            return None
        
        # Setup neighbor search for SmallStructure
        # Use min_dist=0.01 in find_site_neighbors to filter zero-distance bonds
        ns = gemmi.NeighborSearch(small_struct, max_bond_distance).populate()
        
        # Build bond data in GSAS-II format
        bond_data = {
            'bond_label_1': [],
            'bond_label_2': [],
            'bond_distance': [],
            'bond_symmetry_2': []
        }
        
        bond_count = 0
        atom_type_pairs = set()
        
        # Debug: Print all sites found
        print(f"  Debug: Found {len(small_struct.sites)} sites in asymmetric unit:")
        for site in small_struct.sites:
            frac = site.fract
            print(f"    {site.label}: {site.element.name} at ({frac.x:.4f}, {frac.y:.4f}, {frac.z:.4f}) occ={site.occ:.2f}")
        
        # CRITICAL: Generate full supercell for proper coordination detection
        # Gemmi's neighbor search works with asymmetric units, but we need all atoms
        # Generate 3x3x3 supercell to ensure we find all neighbors within cutoff
        print(f"  Generating 3×3×3 supercell for coordination analysis...")
        
        supercell_atoms = []
        
        # For each site in asymmetric unit
        for site in small_struct.sites:
            elem_type = site.element.name
            site_label = site.label
            
            # Apply all symmetry operations
            for op in small_struct.spacegroup.operations():
                # Transform fractional coordinates
                transformed_frac = op.apply_to_xyz([site.fract.x, site.fract.y, site.fract.z])
                
                # Apply lattice translations for 3x3x3 supercell
                for i in range(-1, 2):  # -1, 0, 1
                    for j in range(-1, 2):
                        for k in range(-1, 2):
                            # Add lattice translation
                            final_frac = [
                                transformed_frac[0] + i,
                                transformed_frac[1] + j,
                                transformed_frac[2] + k
                            ]
                            
                            # Convert to Cartesian
                            cart_pos = small_struct.cell.orthogonalize(gemmi.Fractional(*final_frac))
                            
                            supercell_atoms.append({
                                'label': site_label,
                                'element': elem_type,
                                'position': cart_pos,
                                'frac': final_frac,
                                'original_site': site
                            })
        
        print(f"  Generated {len(supercell_atoms)} atoms in supercell")
        
        # Now find neighbors by simple distance calculation
        for site in small_struct.sites:
            site_orth = site.orth(small_struct.cell)
            elem1 = site.label
            elem1_type = site.element.name
            
            neighbors_for_site = []
            
            # Check distance to every atom in supercell
            for atom_dict in supercell_atoms:
                distance = atom_dict['position'].dist(site_orth)
                
                # Skip self and atoms too close/far
                if distance < 0.01 or distance > max_bond_distance:
                    continue
                
                neighbors_for_site.append({
                    'label': atom_dict['label'],
                    'element': atom_dict['element'],
                    'position': atom_dict['position'],
                    'distance': distance
                })
            
            # Deduplicate by position with 0.05Å tolerance  
            # Looser tolerance handles general positions in R-3m better
            unique_neighbors = []
            seen_positions = set()
            
            for neighbor in neighbors_for_site:
                # Round to 0.05Å (50 pm) - catches numerical precision while keeping distinct atoms
                pos_key = (
                    round(neighbor['position'].x / 0.05) * 0.05,
                    round(neighbor['position'].y / 0.05) * 0.05,
                    round(neighbor['position'].z / 0.05) * 0.05
                )
                if pos_key not in seen_positions:
                    seen_positions.add(pos_key)
                    unique_neighbors.append(neighbor)
            
            print(f"  Debug: Site {site.label} ({site.element.name}) has {len(unique_neighbors)} neighbors in supercell")
            
            for neighbor in unique_neighbors:
                elem2 = neighbor['label']
                elem2_type = neighbor['element']
                distance = neighbor['distance']
                atom_type_pairs.add(tuple(sorted([elem1_type, elem2_type])))
                
                # Generate position key with 0.05Å grid snapping (matches deduplication)
                pos_x = round(neighbor['position'].x / 0.05) * 0.05
                pos_y = round(neighbor['position'].y / 0.05) * 0.05
                pos_z = round(neighbor['position'].z / 0.05) * 0.05
                position_key = f"{pos_x:.2f}_{pos_y:.2f}_{pos_z:.2f}"
                symmetry_code = "supercell"
                
                print(f"    -> {elem2} ({elem2_type}) dist={distance:.4f}")
                
                # Store bond
                bond_data['bond_label_1'].append(elem1)
                bond_data['bond_label_2'].append(elem2)
                bond_data['bond_distance'].append(f"{distance:.5f}")
                bond_data['bond_symmetry_2'].append(f"{symmetry_code}|{position_key}")
                bond_count += 1
        
        print(f"  ✓ Calculated {bond_count} bonds using crystallographic symmetry (gemmi)")
        print(f"  ✓ Bond types found: {sorted(atom_type_pairs)}")
        print(f"  ✓ Space group: {small_struct.spacegroup_hm}")
        
        return bond_data
        
    except Exception as e:
        print(f"  ✗ Error calculating bonds with gemmi: {e}")
        import traceback
        traceback.print_exc()
        return None

def is_bond_data_present(data_block):
    """
    Check if a CIF data block contains pre-calculated bond distance data.
    
    Args:
        data_block: PyCifRW data block dictionary
    
    Returns:
        True if bond data present (GSAS-II style), False if missing (ICSD style)
    """
    required_keys = [
        '_geom_bond_atom_site_label_1',
        '_geom_bond_atom_site_label_2',
        '_geom_bond_distance',
        '_geom_bond_site_symmetry_2'
    ]
    
    return all(key in data_block for key in required_keys)

# ============================================================================
# DIRECTORY SELECTION DIALOG
# ============================================================================

def select_directory():
    """
    Opens a tkinter dialog to select a directory containing CIF files.
    Returns the selected directory path or None if cancelled.
    """
    import tkinter as tk
    from tkinter import filedialog
    
    root = tk.Tk()
    root.overrideredirect(True)
    root.attributes('-topmost', True)
    root.withdraw()
    
    print("\n" + "="*80)
    print("SELECT DIRECTORY CONTAINING CIF FILES")
    print("="*80)
    print("A file dialog will open. Please select the directory containing your CIF files.")
    print("(If the dialog appears behind other windows, check your taskbar)")
    
    directory_path = filedialog.askdirectory(
        title="Select Directory Containing CIF Files",
        mustexist=True
    )
    
    root.destroy()
    
    if not directory_path:
        print("\n✗ No directory selected. Exiting.")
        return None
    
    print(f"\n✓ Selected directory: {directory_path}")
    
    # Validate directory contains CIF files
    cif_files = list(Path(directory_path).glob("*.cif"))
    if not cif_files:
        print(f"\n✗ ERROR: No CIF files found in {directory_path}")
        print("Please select a directory containing .cif files")
        return None
    
    print(f"✓ Found {len(cif_files)} CIF files")
    
    return directory_path

# ============================================================================
# CIF PARSING (From Skuld 1.5 - EXACT PRESERVATION)
# ============================================================================

def parse_cif_file(file_path):
    """
    Parse a CIF file and return the CifFile object.
    For ICSD CIFs without bond data, calculates bonds from atomic positions.
    """
    try:
        cf = ReadCif(file_path)
        
        # Check if bonds need to be calculated (ICSD CIFs)
        for block_name, data_block in cf.items():
            if not is_bond_data_present(data_block):
                print(f"  ⚠ No bond data in CIF - calculating from crystallographic symmetry (ICSD mode)...")
                
                # Calculate bonds using ASE
                bond_data = calculate_bonds_from_positions(file_path)
                
                if bond_data is not None:
                    # Add calculated bonds to the data block
                    data_block['_geom_bond_atom_site_label_1'] = bond_data['bond_label_1']
                    data_block['_geom_bond_atom_site_label_2'] = bond_data['bond_label_2']
                    data_block['_geom_bond_distance'] = bond_data['bond_distance']
                    data_block['_geom_bond_site_symmetry_2'] = bond_data['bond_symmetry_2']
                else:
                    print(f"  ✗ Failed to calculate bonds - structure will be skipped")
        
        return cf
    except Exception as e:
        print(f"  ERROR parsing {os.path.basename(file_path)}: {e}")
        return None

def extract_initial_site_data(cif_object):
    """
    EXACT COPY from Skuld 1.5 (lines 808-938)
    Extracts initial atom site data and all bond information,
    including proper handling of symmetry operations for bonds beyond unit cell.
    """
    if not cif_object:
        return {}

    processed_data = {}

    for block_name, data_block in cif_object.items():
        block_data = {
            'unit_cell': {},
            'atom_sites_data': [],
            'all_bond_pairs_found': set(),
            'sites_with_bonds': {}
        }
        processed_data[block_name] = block_data

        # --- General Data Items ---
        block_data['publ_author_name'] = data_block.get("_publ_contact_author_name")
        block_data['gsasii_version'] = data_block.get("_gsas_GSASII_version")
        block_data['pd_block_id'] = data_block.get("_pd_block_id")

        # --- Atom Site Data Processing ---
        label_to_canonical_coords = {} 
        label_to_original_coord_strs = {}
        label_to_type = {}
        
        try:
            labels = data_block['_atom_site_label']
            types = data_block['_atom_site_type_symbol']
            xs_orig = data_block['_atom_site_fract_x'] 
            ys_orig = data_block['_atom_site_fract_y']
            zs_orig = data_block['_atom_site_fract_z']

            if not (labels and types and xs_orig and ys_orig and zs_orig and \
                    len(labels) == len(types) == len(xs_orig) == len(ys_orig) == len(zs_orig)):
                print(f"  No complete _atom_site_loop data found in block {block_name}. Skipping atom site processing for this block.")
                block_data['atom_sites_data'] = [] 
            else:
                for i in range(len(labels)):
                    label = labels[i].strip()
                    atom_type_raw = types[i].strip().rstrip('+-0123456789')
                    atom_type = atom_type_raw.capitalize() if len(atom_type_raw) > 1 else atom_type_raw.upper()
                    
                    try:
                        x = float(xs_orig[i].split('(')[0].strip())
                        y = float(ys_orig[i].split('(')[0].strip())
                        z = float(zs_orig[i].split('(')[0].strip())
                        coords_tuple = (x, y, z)
                        
                        block_data['atom_sites_data'].append({'label': label, 'type': atom_type, 'coords': coords_tuple})
                        label_to_canonical_coords[label] = coords_tuple 
                        label_to_original_coord_strs[label] = (xs_orig[i], ys_orig[i], zs_orig[i])
                        label_to_type[label] = atom_type 
                    except (ValueError, IndexError):
                        print(f"  Warning: Could not parse coordinates for atom {label}. Skipping.")
                        continue
        except KeyError:
            print(f"  No _atom_site_label or related loop data found in block {block_name}. Skipping atom site processing for this block.")
            block_data['atom_sites_data'] = []

        # --- Bond Distances (CRITICAL: includes symmetry code handling) ---
        all_raw_bonds_per_central_site = {} 
        site_metadata = {} 

        bond_label_1s = data_block.get("_geom_bond_atom_site_label_1")
        bond_label_2s = data_block.get("_geom_bond_atom_site_label_2")
        bond_distance_raw_strs = data_block.get("_geom_bond_distance")
        bond_symmetry_2s = data_block.get("_geom_bond_site_symmetry_2")

        if bond_label_1s and bond_label_2s and bond_distance_raw_strs and bond_symmetry_2s and \
           all(len(lst) == len(bond_label_1s) for lst in [bond_label_2s, bond_distance_raw_strs, bond_symmetry_2s]):
            
            for i in range(len(bond_label_1s)):
                atom1_label = bond_label_1s[i].strip()
                atom2_label = bond_label_2s[i].strip()
                dist_str = bond_distance_raw_strs[i].strip()
                atom2_symmetry_code = bond_symmetry_2s[i].strip()  # CRITICAL: preserves symmetry info

                dist_val, dist_err = parse_value_with_error(dist_str)
                
                if dist_val is None:
                    continue 

                atom1_canonical_coords = label_to_canonical_coords.get(atom1_label)
                atom2_canonical_coords = label_to_canonical_coords.get(atom2_label) 

                atom1_type = label_to_type.get(atom1_label)
                atom2_type = label_to_type.get(atom2_label)

                # CRITICAL FIX for ICSD CIFs: If type not found in label_to_type,
                # extract element from label (e.g., "Na1" -> "Na", "Cr1" -> "Cr")
                if not atom1_type and atom1_label:
                    # Extract element symbol from label (strip trailing digits)
                    import re
                    match = re.match(r'^([A-Z][a-z]?)', atom1_label)
                    if match:
                        atom1_type = match.group(1)
                
                if not atom2_type and atom2_label:
                    import re
                    match = re.match(r'^([A-Z][a-z]?)', atom2_label)
                    if match:
                        atom2_type = match.group(1)

                if not atom1_type or not atom2_type:
                    continue 
                
                # Validation
                if not validate_element_combination(atom1_type, atom2_type):
                    print(f"  Warning: Unusual element combination {atom1_type}-{atom2_type}, but including in analysis")
                
                block_data['all_bond_pairs_found'].add(tuple(sorted((atom1_type, atom2_type))))

                central_atom_coords = atom1_canonical_coords
                central_atom_element = atom1_type
                
                if central_atom_coords:
                    if central_atom_coords not in all_raw_bonds_per_central_site:
                        all_raw_bonds_per_central_site[central_atom_coords] = []
                        site_metadata[central_atom_coords] = {
                            'elements': set(),
                            'labels_at_site': set(),
                        }

                    site_metadata[central_atom_coords]['elements'].add(central_atom_element)
                    site_metadata[central_atom_coords]['labels_at_site'].add(atom1_label)
                    
                    # CRITICAL: Store complete bond tuple including symmetry code
                    all_raw_bonds_per_central_site[central_atom_coords].append(
                        (atom1_label, atom2_label, atom1_type, atom2_type, atom2_symmetry_code, dist_val, dist_err)
                    )
        else:
            print(f"  No geometric bond data found or incomplete in block {block_name}.")

        block_data['sites_with_bonds'] = {
            coords: {
                'elements': sorted(list(site_metadata[coords]['elements'])),
                'labels': sorted(list(site_metadata[coords]['labels_at_site'])),
                'raw_bonds_list': raw_bonds_list
            } for coords, raw_bonds_list in all_raw_bonds_per_central_site.items()
        }
    
    return processed_data

# ============================================================================
# AUTOMATED BOND SELECTION (v1.2 - Multi-anion support)
# ============================================================================

def auto_select_bond_types(processed_data, target_metals=['Na', 'K', 'Li', 'Cr', 'Ni', 'Mn', 'Fe', 'Ti', 'Sb', 'Mg', 'Al', 'Ca', 'Sr', 'Ba', 'Zn', 'Cd', 'Cu', 'Co', 'V', 'Ga', 'In', 'Sn']):
    """
    Automatically select metal-anion bond types for analysis.
    Handles all anion types (O, F, Cl, Br, I, S, Se, Te, etc.), not just oxides.
    
    Args:
        processed_data: Data structure from previous processing steps
        target_metals: List of metals to prioritize
    
    Returns: processed_data with 'selected_bond_types_for_analysis' added
    """
    for block_name, block_content in processed_data.items():
        unique_bond_pairs = sorted(list(block_content['all_bond_pairs_found']))
        
        print(f"  DEBUG: Found {len(unique_bond_pairs)} unique bond pairs: {unique_bond_pairs}")
        
        if not unique_bond_pairs:
            block_content['selected_bond_types_for_analysis'] = {}
            continue
        
        # Auto-select M-X bonds (where X is any anion)
        selected_bond_types_map = {}
        
        for type1, type2 in unique_bond_pairs:
            # Identify which element is the anion
            anion = None
            cation = None
            
            if is_anion(type1) and not is_anion(type2):
                anion = type1
                cation = type2
            elif is_anion(type2) and not is_anion(type1):
                anion = type2
                cation = type1
            else:
                # Neither is a known anion, or both are - skip
                continue
            
            # Check if cation is a metal
            cat_category = db_manager.get_element_category(cation)
            is_target = cation in target_metals
            # BUGFIX: Changed 'poor_metals' to 'post_transition_metals'
            is_metal = cat_category in ['alkali_metals', 'alkaline_earth_metals', 'transition_metals_3d', 
                                       'transition_metals_4d', 'transition_metals_5d', 'lanthanides', 
                                       'actinides', 'post_transition_metals']  # ← FIXED!
            
            print(f"  DEBUG: Bond {cation}-{anion}: target={is_target}, metal_category={cat_category}, is_metal={is_metal}")
            
            if is_target or is_metal:
                # Cation is central, anion is ligand
                selected_bond_types_map[(cation, anion)] = {
                    'central_atom_type': cation,
                    'ligand_type': anion
                }
        
        block_content['selected_bond_types_for_analysis'] = selected_bond_types_map
        
        if selected_bond_types_map:
            print(f"  Auto-selected {len(selected_bond_types_map)} M-X bond types: {list(selected_bond_types_map.keys())}")
        else:
            print(f"  No M-X bonds found for auto-selection")
    
    return processed_data

# ============================================================================
# AUTOMATED COORDINATION ANALYSIS (Replaces interactive_review_and_correct)
# ============================================================================

def auto_analyze_coordination(processed_data):
    """
    Automatically analyze coordination numbers and bond lengths.
    Uses the EXACT gap analysis logic from Skuld 1.5 but accepts results automatically.
    
    CRITICAL: Preserves lines 1083-1160 from Skuld 1.5 for proper coordination detection.
    """
    final_data = processed_data

    for block_name, block_content in final_data.items():
        
        if not block_content['sites_with_bonds'] or not block_content['selected_bond_types_for_analysis']:
            continue

        for coords, site_info in block_content['sites_with_bonds'].items():
            central_atom_elements_at_site = site_info['elements']
            
            bonds_for_analysis = {}
            for (central_type_selected, ligand_type_selected) in block_content['selected_bond_types_for_analysis'].keys():
                if central_type_selected in central_atom_elements_at_site:
                    filtered_raw_bonds = [
                        b for b in site_info['raw_bonds_list'] 
                        if b[2] == central_type_selected and b[3] == ligand_type_selected
                    ]
                    if filtered_raw_bonds:
                        bonds_for_analysis[(central_type_selected, ligand_type_selected)] = filtered_raw_bonds

            if not bonds_for_analysis:
                continue

            site_info['cn_and_bonds_by_type'] = {}

            for (central_type, ligand_type), raw_bonds_list_for_pair in bonds_for_analysis.items():
                
                # Report site coordinates for validation
                print(f"\n    Analyzing site at {coords}: {central_type}")
                
                # ======================================================================
                # CRITICAL SECTION: EXACT COPY from Skuld 1.5 (lines 1083-1160)
                # This handles symmetry operations and proper coordination detection
                # ======================================================================
                
                # STEP 1: Identify unique geometric positions using (label, position)
                # For ICSD files, symmetry_code now includes |x_y_z position encoding
                unique_geometric_positions_to_shortest_bond = {}
                
                # DEBUG: Show first few bonds to verify position encoding
                if len(raw_bonds_list_for_pair) <= 10:
                    print(f"  DEBUG: Raw bonds for {central_type}-{ligand_type}:")
                    for bond_tuple in raw_bonds_list_for_pair[:10]:
                        print(f"    {bond_tuple[0]}-{bond_tuple[1]}: {bond_tuple[5]:.4f}Å symm='{bond_tuple[4]}'")
                
                # DEBUG: Track unique positions
                position_debug = set()
                
                for bond_tuple in raw_bonds_list_for_pair:
                    ligand_label = bond_tuple[1]
                    ligand_symmetry_code = bond_tuple[4]  # May include |x_y_z position
                    dist_val = bond_tuple[5]
                    dist_err = bond_tuple[6]

                    # Use Cartesian position if available, else use symmetry code
                    if '|' in ligand_symmetry_code:
                        # ICSD mode: extract position part for uniqueness
                        symm_part, pos_part = ligand_symmetry_code.split('|', 1)
                        geo_position_id = (ligand_label, pos_part)  # Use 3D position
                        position_debug.add(pos_part)
                    else:
                        # GSAS-II mode: use symmetry code
                        geo_position_id = (ligand_label, ligand_symmetry_code)
                    
                    if geo_position_id not in unique_geometric_positions_to_shortest_bond or \
                       dist_val < unique_geometric_positions_to_shortest_bond[geo_position_id][0]:  # Compare with OLD distance [0], not error [1]
                        unique_geometric_positions_to_shortest_bond[geo_position_id] = (dist_val, dist_err)
                
                # DEBUG: Show unique positions found
                if position_debug:
                    print(f"  DEBUG: Found {len(position_debug)} unique positions: {sorted(list(position_debug)[:6])}")

                # STEP 2: Create list of all bonds within coordination sphere
                cn_bonds_list = []
                for (dist_val, dist_err) in unique_geometric_positions_to_shortest_bond.values():
                    cn_bonds_list.append((dist_val, dist_err))
                
                sorted_cn_bonds = sorted(cn_bonds_list, key=lambda x: x[0])

                # STEP 3: Gap analysis to find primary coordination sphere
                # Add minimum gap threshold to avoid splitting coordination due to numerical precision
                MIN_GAP_THRESHOLD = 0.05  # Å - ignore gaps smaller than this
                initial_cn = 0
                primary_sphere_cutoff_dist = 0.0 
                
                if sorted_cn_bonds:
                    max_gap = 0
                    cutoff_index = len(sorted_cn_bonds) 
                    
                    if len(sorted_cn_bonds) > 1:
                        for i in range(len(sorted_cn_bonds) - 1):
                            gap = sorted_cn_bonds[i+1][0] - sorted_cn_bonds[i][0]
                            # Only consider gaps larger than threshold
                            if gap > MIN_GAP_THRESHOLD and gap > max_gap:
                                max_gap = gap
                                cutoff_index = i + 1 
                    
                    initial_cn = cutoff_index
                    if cutoff_index > 0:
                        primary_sphere_cutoff_dist = sorted_cn_bonds[cutoff_index - 1][0]

                # STEP 4: Extract bonds within primary sphere
                initial_bonds_for_display = []
                for bond_tuple in sorted_cn_bonds: 
                    dist_val, dist_err = bond_tuple
                    if dist_val <= primary_sphere_cutoff_dist:
                        initial_bonds_for_display.append({'distance': dist_val, 'error': dist_err})
                
                # STEP 5: Count bond multiplicities
                initial_detailed_bond_counts = []
                if initial_bonds_for_display:
                    grouped_bonds_info = []
                    for bond_dict in initial_bonds_for_display:
                        rounded_dist = round(bond_dict['distance'], 5)
                        original_error = bond_dict['error']
                        grouped_bonds_info.append((rounded_dist, original_error, bond_dict['distance']))

                    bond_counts_map = Counter((r_d, o_e) for r_d, o_e, _ in grouped_bonds_info)
                    sorted_unique_keys = sorted(bond_counts_map.keys())

                    for (dist_val_rounded, dist_err_val) in sorted_unique_keys:
                        count = bond_counts_map[(dist_val_rounded, dist_err_val)]
                        original_dist_val = next((orig_d for r_d, e_v, orig_d in grouped_bonds_info 
                                                  if r_d == dist_val_rounded and e_v == dist_err_val), 
                                                 dist_val_rounded)

                        initial_detailed_bond_counts.append({
                            'count': count, 
                            'distance': original_dist_val, 
                            'error': dist_err_val 
                        })

                # ======================================================================
                # END CRITICAL SECTION
                # ======================================================================
                
                # AUTOMATED DECISION: Accept calculated CN and bonds
                confirmed_cn = initial_cn
                confirmed_bond_counts = initial_detailed_bond_counts
                
                # DIAGNOSTIC OUTPUT: Show bond distances for validation
                if initial_detailed_bond_counts:
                    bond_strs = []
                    for b in initial_detailed_bond_counts:
                        bond_strs.append(f"{b['count']}×{b['distance']:.4f}Å")
                    
                    avg_dist = sum(b['distance']*b['count'] for b in initial_detailed_bond_counts) / sum(b['count'] for b in initial_detailed_bond_counts)
                    
                    # VALIDATION: Check against expected values for different bond types
                    warnings = []
                    
                    # Oxide validation
                    if ligand_type == 'O':
                        if central_type == 'Cr' and confirmed_cn == 6:
                            if avg_dist < 1.95 or avg_dist > 2.05:
                                warnings.append(f"⚠️ Cr-O distance {avg_dist:.3f}Å unusual (expect 1.98-2.01Å)")
                        elif central_type == 'Na' and confirmed_cn == 6:
                            if avg_dist < 2.25 or avg_dist > 2.40:
                                warnings.append(f"⚠️ Na-O distance {avg_dist:.3f}Å unusual (expect 2.30-2.35Å)")
                        elif central_type == 'Ni' and confirmed_cn == 6:
                            if avg_dist < 2.00 or avg_dist > 2.15:
                                warnings.append(f"⚠️ Ni-O distance {avg_dist:.3f}Å unusual (expect 2.05-2.10Å)")
                    
                    # Halide validation
                    elif ligand_type == 'Cl':
                        if central_type == 'Na' and confirmed_cn == 6:
                            if avg_dist < 2.75 or avg_dist > 2.90:
                                warnings.append(f"⚠️ Na-Cl distance {avg_dist:.3f}Å unusual (expect 2.80-2.85Å)")
                    
                    elif ligand_type == 'F':
                        if central_type == 'Na' and confirmed_cn == 6:
                            if avg_dist < 2.25 or avg_dist > 2.45:
                                warnings.append(f"⚠️ Na-F distance {avg_dist:.3f}Å unusual (expect 2.30-2.35Å)")
                    
                    # Sulfide validation
                    elif ligand_type == 'S':
                        if central_type == 'Na' and confirmed_cn == 6:
                            if avg_dist < 2.80 or avg_dist > 3.00:
                                warnings.append(f"⚠️ Na-S distance {avg_dist:.3f}Å unusual (expect 2.85-2.95Å)")
                    
                    # Print diagnostic info
                    print(f"    {central_type}-{ligand_type}: CN={confirmed_cn}, bonds=[{', '.join(bond_strs)}], avg={avg_dist:.4f}Å")
                    for warning in warnings:
                        print(f"      {warning}")
                
                site_info['cn_and_bonds_by_type'][(central_type, ligand_type)] = {
                    'confirmed_cn': confirmed_cn,
                    'confirmed_bond_counts': confirmed_bond_counts
                }
    
    return final_data

# ============================================================================
# AUTOMATED VALENCE ASSIGNMENT (Replaces get_valences_interactively)
# ============================================================================

def auto_assign_valences(processed_data, user_oxidation_states=None):
    """
    Automatically assign oxidation states using user-provided overrides or database defaults.
    
    Args:
        processed_data: Data structure from previous processing steps
        user_oxidation_states: Optional dict of {element: valence} from interactive prompt
    
    Uses user-provided oxidation states if available, otherwise falls back to database.
    """
    for block_name, block_content in processed_data.items():
        if not block_content['sites_with_bonds'] or not block_content.get('selected_bond_types_for_analysis'):
            continue

        for coords, site_info in block_content['sites_with_bonds'].items():
            if 'cn_and_bonds_by_type' not in site_info:
                continue
            
            relevant_elements = set()
            for (central_type, ligand_type) in site_info['cn_and_bonds_by_type'].keys():
                relevant_elements.add(central_type)
                relevant_elements.add(ligand_type)
            
            if not relevant_elements:
                continue

            site_valences = {}
            for element in relevant_elements:
                # PRIORITY 1: User-provided oxidation states
                if user_oxidation_states and element in user_oxidation_states:
                    valence = user_oxidation_states[element]
                
                # PRIORITY 2: Database common states
                else:
                    common_states = db_manager.get_common_oxidation_states(element)
                    
                    if common_states:
                        # Use most common state (first in list)
                        valence = common_states[0]
                    else:
                        # PRIORITY 3: Hardcoded fallback defaults
                        default_valences = {
                            'Na': 1, 'K': 1, 'Li': 1,
                            'O': -2, 'F': -1, 'S': -2,
                            'Ni': 2, 'Mn': 3, 'Fe': 3, 'Cr': 3,
                            'Ti': 4, 'Al': 3, 'Mg': 2, 'Sb': 5
                        }
                        valence = default_valences.get(element, 0)
                
                site_valences[element] = valence
            
            site_info['confirmed_valences'] = site_valences
    
    return processed_data

# ============================================================================
# BVS CALCULATIONS (From Skuld 1.5 - EXACT PRESERVATION)
# ============================================================================

def perform_calculations_and_report(processed_data, b_param_default=0.37):
    """
    Perform BVS, Shannon-Prewitt, and other crystallochemical calculations.
    PRESERVES EXACT calculation logic from Skuld 1.5 (lines 1307-1537).
    """
    calculated_results = {}

    for block_name, block_content in processed_data.items():
        
        if not block_content['sites_with_bonds'] or not block_content.get('selected_bond_types_for_analysis'):
            continue
        
        calculated_results[block_name] = {}

        for coords, site_info in block_content['sites_with_bonds'].items():
            if 'cn_and_bonds_by_type' not in site_info:
                continue

            site_results = {
                'elements': site_info['elements'],
                'labels': site_info['labels'],
                'element_data': {}
            }

            for (central_type, ligand_type), bond_analysis_data in site_info['cn_and_bonds_by_type'].items():
                confirmed_cn = bond_analysis_data['confirmed_cn']
                confirmed_bond_counts = bond_analysis_data['confirmed_bond_counts']
                
                central_valence = site_info['confirmed_valences'].get(central_type)
                ligand_valence = site_info['confirmed_valences'].get(ligand_type)

                if central_valence is None or ligand_valence is None:
                    continue
                
                element_calc_data = {'valence': central_valence, 'ligand_valence': ligand_valence, 'cn': confirmed_cn}

                # --- Shannon-Prewitt Ionic Radii Comparison ---
                if confirmed_bond_counts:
                    central_radius = db_manager.get_ionic_radius(central_type, central_valence, confirmed_cn)
                    ligand_radius = db_manager.get_ionic_radius(ligand_type, ligand_valence, confirmed_cn)

                    if central_radius is not None and ligand_radius is not None:
                        expected_distance = central_radius + ligand_radius
                        
                        element_shannon_diffs = []
                        
                        for bond_info in confirmed_bond_counts:
                            count = bond_info['count']
                            observed_distance = bond_info['distance']
                            bond_error = bond_info.get('error', np.nan)
                            percent_diff = ((observed_distance - expected_distance) / expected_distance) * 100
                            element_shannon_diffs.append({
                                'bond_count': count, 
                                'observed_distance': observed_distance, 
                                'expected_distance': expected_distance, 
                                'percent_diff': percent_diff,
                                'error': bond_error 
                            })
                        
                        element_calc_data['shannon_diffs'] = element_shannon_diffs
                        element_calc_data['cation_radius'] = central_radius
                        element_calc_data['anion_radius'] = ligand_radius

                # --- Ionic Potential ---
                if element_calc_data.get('cation_radius') and element_calc_data['cation_radius'] > 0:
                    ionic_potential = float(abs(central_valence)) / element_calc_data['cation_radius']
                    element_calc_data['ionic_potential'] = ionic_potential

                # --- Bond Valence Sum (BVS) Calculation ---
                if confirmed_bond_counts:
                    r0_val, b_val = db_manager.get_bv_parameters(central_type, central_valence, ligand_type)
                    
                    if r0_val is not None:
                        bvs_total = 0.0
                        
                        for bond_info in confirmed_bond_counts:
                            count = bond_info['count']
                            distance = bond_info['distance']
                            
                            s = math.exp((r0_val - distance) / b_val)
                            bvs_total += (s * count)
                        
                        if central_valence != 0:
                            bvs_percent_diff = ((bvs_total - abs(central_valence)) / abs(central_valence)) * 100
                            
                            element_calc_data['bvs'] = bvs_total
                            element_calc_data['bvs_percent_diff'] = bvs_percent_diff

                # --- Bond Length Distortion (BLD) ---
                if confirmed_bond_counts and confirmed_cn > 0:
                    total_distance_for_avg = sum(b['distance'] * b['count'] for b in confirmed_bond_counts)
                    total_count_for_avg = sum(b['count'] for b in confirmed_bond_counts)
                    
                    if total_count_for_avg > 0:
                        average_observed_distance = total_distance_for_avg / total_count_for_avg
                        
                        sum_of_squared_deviations = 0.0
                        for bond_info in confirmed_bond_counts:
                            distance = bond_info['distance']
                            count = bond_info['count']
                            sum_of_squared_deviations += count * ((distance - average_observed_distance) / average_observed_distance)**2
                        
                        bld = sum_of_squared_deviations / confirmed_cn
                        element_calc_data['bond_length_distortion'] = bld

                site_results['element_data'][central_type] = element_calc_data
            
            calculated_results[block_name][coords] = site_results
            
    return calculated_results

def calculate_global_instability_index(calculated_results):
    """
    Calculate Global Instability Index (GII) - EXACT from Skuld 1.5 (lines 671-776)
    """
    gii_results = {}
    
    for block_name, block_results in calculated_results.items():
        total_bvs_deviation = 0.0
        total_formal_valence = 0.0
        site_contributions = []
        
        for site_coords, site_data in block_results.items():
            if 'element_data' not in site_data:
                continue
                
            for element in site_data['elements']:
                element_data = site_data['element_data'].get(element, {})
                
                if 'bvs' in element_data and element_data['bvs'] is not None:
                    observed_bvs = element_data['bvs']
                    formal_valence = abs(element_data.get('valence', 0))
                    
                    if formal_valence > 0:
                        bvs_deviation = abs(observed_bvs - formal_valence)
                        total_bvs_deviation += bvs_deviation
                        total_formal_valence += formal_valence
                        
                        site_contributions.append({
                            'site': site_coords,
                            'element': element,
                            'formal_valence': formal_valence,
                            'observed_bvs': observed_bvs,
                            'bvs_deviation': bvs_deviation,
                            'bvs_percent_diff': element_data.get('bvs_percent_diff', 0),
                            'element_category': db_manager.get_element_category(element)
                        })
        
        if total_formal_valence > 0:
            gii = total_bvs_deviation / total_formal_valence
            
            if gii < 0.05:
                stability_assessment = "Very Stable"
            elif gii < 0.1:
                stability_assessment = "Stable" 
            elif gii < 0.2:
                stability_assessment = "Moderately Unstable"
            elif gii < 0.3:
                stability_assessment = "Unstable"
            else:
                stability_assessment = "Highly Unstable"
            
            gii_results[block_name] = {
                'gii': gii,
                'stability_assessment': stability_assessment,
                'site_contributions': site_contributions,
                'total_sites_analyzed': len(site_contributions)
            }
        else:
            gii_results[block_name] = {
                'gii': None,
                'stability_assessment': 'Cannot Assess',
                'site_contributions': [],
                'total_sites_analyzed': 0
            }
    
    return gii_results

# ============================================================================
# BATCH PROCESSING ENGINE
# ============================================================================

def process_single_cif(file_path, user_oxidation_states=None):
    """
    Process a single CIF file using validated Eir 1.0 logic.
    NEW in v1.2 continued: Mixed occupancy detection, Bosi (2014) strain analysis,
    and automatic 3D visualisation of problematic sites.
    
    Args:
        file_path: Path to CIF file
        user_oxidation_states: Optional dict of {element: valence} overrides
                              If provided, these oxidation states will be used instead of database defaults
    
    Returns:
        dict with processing results including success status, metadata, sites data, and GII
    """
    results = {
        'success': False,
        'error': None,
        'metadata': {},
        'sites': [],
        'gii': None,
        'filename': os.path.basename(file_path)
    }
    
    # Parse CIF
    cif_obj = parse_cif_file(file_path)
    if cif_obj is None:
        results['error'] = "Failed to parse CIF file"
        return results
    
    # Process first data block
    block_name = list(cif_obj.keys())[0]
    data_block = cif_obj[block_name]
    
    # Extract metadata
    results['metadata'] = {
        'block_name': block_name,
        'filename': os.path.basename(file_path),
        'chemical_formula': data_block.get("_chemical_formula_sum", "N/A"),
        'space_group': data_block.get("_space_group_name_H-M_alt", 
                                     data_block.get("_symmetry_space_group_name_H-M", "N/A"))
    }
    
    # Extract ICSD publication metadata
    icsd_metadata = extract_icsd_metadata(file_path)
    results['metadata'].update({
        'icsd_code': icsd_metadata.get('icsd_code'),
        'year': icsd_metadata.get('year'),
        'journal': icsd_metadata.get('journal'),
        'authors': ', '.join(icsd_metadata.get('authors', [])) if icsd_metadata.get('authors') else None,
        'r_factor': icsd_metadata.get('r_factor'),
        'density': icsd_metadata.get('density'),
        'measurement_temperature_K': icsd_metadata.get('measurement_temperature_K'),
        'measurement_pressure_Pa': icsd_metadata.get('measurement_pressure_Pa'),
        'measurement_pressure_GPa': icsd_metadata.get('measurement_pressure_GPa'),
        'measurement_conditions': icsd_metadata.get('measurement_conditions')
    })
    
    # Validated Eir 1.0 workflow
    try:
        # Step 1: Extract site data
        initial_extracted_data = extract_initial_site_data(cif_obj)
        
        if not initial_extracted_data:
            results['error'] = "No data extracted from CIF"
            return results
        
        # Step 2: Auto-select bond types
        data_with_selected_bonds = auto_select_bond_types(initial_extracted_data)
        
        if not data_with_selected_bonds[block_name].get('selected_bond_types_for_analysis'):
            results['error'] = "No M-X bonds found"  # Updated to M-X (not just M-O)
            return results
        
        # Step 3: Auto-analyze coordination
        final_corrected_data = auto_analyze_coordination(data_with_selected_bonds)
        
        # Step 4: Auto-assign valences
        final_data_with_valences = auto_assign_valences(final_corrected_data, user_oxidation_states)
        
        # Step 5: Perform calculations
        calculated_results = perform_calculations_and_report(final_data_with_valences)
        
        # Step 6: Calculate GII
        gii_results = calculate_global_instability_index(calculated_results)
        
        # ============================================================================
        # Extract results for export WITH MIXED OCCUPANCY ANALYSIS
        # ============================================================================
        if block_name in calculated_results:
            for coords, site_data in calculated_results[block_name].items():
                # BUGFIX: Match each element to its corresponding label
                elements = site_data.get('elements', [])
                labels = site_data.get('labels', ['unknown'])
                
                for idx, element in enumerate(elements):
                    elem_data = site_data['element_data'].get(element, {})
                    
                    # BUGFIX: Match element to label by index
                    if idx < len(labels):
                        site_label = labels[idx]
                    else:
                        # Fallback: find label that starts with element symbol
                        site_label = next((lbl for lbl in labels if lbl.startswith(element)), labels[0])
                    
                    # Get bond information for 3D visualisation
                    bond_counts = None
                    
                    # Extract bond counts from coordination analysis
                    if block_name in final_corrected_data:
                        for site_coords, site_info in final_corrected_data[block_name]['sites_with_bonds'].items():
                            if site_coords == coords:
                                if 'cn_and_bonds_by_type' in site_info:
                                    for (central_type, ligand_type), bond_analysis in site_info['cn_and_bonds_by_type'].items():
                                        if central_type == element:
                                            bond_counts = bond_analysis.get('confirmed_bond_counts', [])
                                            break
                                
                                # Extract full site_info for mixed occupancy analysis
                                full_site_info = site_info
                                break
                    
                    # ========================================================================
                    # Mixed occupancy analysis (from helper functions in Cell 3)
                    # ========================================================================
                    mixed_occ_data = analyze_mixed_occupancy_at_site(
                        coords, 
                        site_data,  # from calculated_results - has 'elements' list
                        final_corrected_data, 
                        block_name
                    )
                    
                    # ========================================================================
                    # Calculate deviation and d1 (discrepancy factor from Bosi 2014)
                    # ========================================================================
                    bvs_val = elem_data.get('bvs')
                    valence = elem_data.get('valence')
                    
                    # Calculate deviation (same as before)
                    if bvs_val is not None and valence is not None:
                        deviation = bvs_val - abs(valence)
                    else:
                        deviation = None
                    
                    # Calculate d1 and bonding state
                    d1, abs_d1, bonding_state = calculate_discrepancy_factor(bvs_val, valence)
                    
                    # ========================================================================
                    # Assign confidence flag based on disorder type and strain
                    # ========================================================================
                    confidence, bosi_likely, needs_review = assign_confidence_flag(
                        mixed_occ_data['disorder_type'],
                        mixed_occ_data['valence_difference'],
                        abs_d1,
                        deviation
                    )
                    
                    # ========================================================================
                    # Build site result with ALL columns
                    # ========================================================================
                    site_result = {
                        # Original columns (unchanged)
                        'coords': coords,
                        'element': element,
                        'site_label': site_label,
                        'valence': elem_data.get('valence'),
                        'cn': elem_data.get('cn'),
                        'bvs': round(elem_data.get('bvs'), 4) if elem_data.get('bvs') is not None else None,
                        'bvs_percent_diff': round(elem_data.get('bvs_percent_diff'), 2) if elem_data.get('bvs_percent_diff') is not None else None,
                        'deviation': round(deviation, 4) if deviation is not None else None,
                        
                        # Mixed occupancy columns
                        'has_mixed_occupancy': mixed_occ_data['has_mixed_occupancy'],
                        'occupancy_string': mixed_occ_data['occupancy_string'],
                        'n_elements_at_site': mixed_occ_data['n_elements_at_site'],
                        'disorder_type': mixed_occ_data['disorder_type'],
                        'valence_difference': round(mixed_occ_data['valence_difference'], 1) if mixed_occ_data['valence_difference'] else 0.0,
                        'max_valence_at_site': mixed_occ_data['max_valence_at_site'],
                        'min_valence_at_site': mixed_occ_data['min_valence_at_site'],
                        
                        # Bosi (2014) strain indicators
                        'discrepancy_d1': round(d1, 4) if d1 is not None else None,
                        'abs_discrepancy_d1': round(abs_d1, 4) if abs_d1 is not None else None,
                        'bonding_state': bonding_state,
                        
                        # Confidence & flagging
                        'confidence_flag': confidence,
                        'bosi_artifact_likely': bosi_likely,
                        'requires_manual_review': needs_review,
                        
                        # Original columns (unchanged)
                        'ionic_potential': round(elem_data.get('ionic_potential'), 3) if elem_data.get('ionic_potential') is not None else None,
                        'cation_radius': round(elem_data.get('cation_radius'), 3) if elem_data.get('cation_radius') is not None else None,
                        'bld': round(elem_data.get('bond_length_distortion'), 6) if elem_data.get('bond_length_distortion') is not None else None
                    }
                    
                    # Add Shannon-Prewitt data if available (unchanged)
                    if 'shannon_diffs' in elem_data and elem_data['shannon_diffs']:
                        avg_shannon_diff = np.mean([s['percent_diff'] for s in elem_data['shannon_diffs']])
                        site_result['shannon_diff'] = round(avg_shannon_diff, 2)
                        site_result['avg_bond_length'] = round(np.mean([s['observed_distance'] for s in elem_data['shannon_diffs']]), 4)
                    
                    # ========================================================================
                    # 3D visualisation for problematic sites
                    # ========================================================================
                    if deviation is not None and abs(deviation) > 0.30 and PY3DMOL_AVAILABLE and bond_counts:
                        print(f"\n    ⚠️  Significant BVS deviation detected for {element} site!")
                        print(f"        BVS: {bvs_val:.3f}  │  Expected: {valence:+d}  │  Δ: {deviation:+.3f} v.u.")
                        
                        # NEW: Print diagnostic information
                        if d1 is not None:
                            print(f"        d1: {d1:+.3f} v.u. ({bonding_state})")
                        print(f"        Disorder: {mixed_occ_data['disorder_type']}", end="")
                        if mixed_occ_data['valence_difference'] > 0:
                            print(f" (ΔV={mixed_occ_data['valence_difference']:.0f})")
                        else:
                            print()
                        print(f"        Occupancy: {mixed_occ_data['occupancy_string']}")
                        print(f"        Confidence: {confidence}")
                        if bosi_likely:
                            print(f"        ⚠️  Bosi artifact likely - spectroscopy required!")
                        
                        print(f"    [Generating 3D visualisation...]")
                        
                        try:
                            visualize_problematic_site_3d(
                                cif_path=file_path,
                                element=element,
                                site_label=site_label,
                                bonds_info=bond_counts,
                                deviation=deviation,
                                formal_charge=valence,
                                bvs_value=bvs_val,
                                filename=os.path.basename(file_path)
                            )
                        except Exception as viz_error:
                            print(f"    [3D visualisation failed: {viz_error}]")
                    
                    elif deviation is not None and abs(deviation) > 0.30 and not PY3DMOL_AVAILABLE:
                        print(f"\n    ⚠️  Significant deviation ({deviation:+.3f} v.u.)")
                        if d1 is not None:
                            print(f"        d1: {d1:+.3f} v.u. ({bonding_state})")
                        print(f"        Disorder: {mixed_occ_data['disorder_type']}")
                        print(f"        Confidence: {confidence}")
                        print(f"        Install py3Dmol for 3D visualisation")
                    
                    results['sites'].append(site_result)
        
        # Add GII (unchanged)
        if block_name in gii_results:
            results['gii'] = gii_results[block_name].get('gii')
            results['stability_assessment'] = gii_results[block_name].get('stability_assessment')
        
        results['success'] = True
        
    except Exception as e:
        results['error'] = f"Processing error: {str(e)}"
        import traceback
        print(f"  Detailed error: {traceback.format_exc()}")
    
    return results

# ============================================================================
# INTERACTIVE OXIDATION STATE ASSIGNMENT
# ============================================================================

def collect_unique_elements_from_cifs(directory_path):
    """
    Pre-scan all CIF files to identify unique elements present.
    This allows asking for oxidation states once before batch processing.
    
    Args:
        directory_path: Path to directory containing CIF files
    
    Returns:
        set of element symbols found across all CIFs
    """
    cif_files = list(Path(directory_path).glob("*.cif"))
    unique_elements = set()
    
    print("\n" + "="*80)
    print("PRE-SCANNING CIF FILES TO IDENTIFY ELEMENTS")
    print("="*80)
    
    for cif_file in cif_files:
        try:
            cf = ReadCif(str(cif_file))
            
            for block_name, data_block in cf.items():
                # Get element types from atom site data
                types = data_block.get('_atom_site_type_symbol', [])
                
                for atom_type_raw in types:
                    # Clean up: "Na1" → "Na", "O2-" → "O"
                    atom_type = atom_type_raw.strip().rstrip('+-0123456789')
                    atom_type = atom_type.capitalize() if len(atom_type) > 1 else atom_type.upper()
                    unique_elements.add(atom_type)
        
        except Exception as e:
            continue  # Skip problematic files in pre-scan
    
    print(f"✓ Found {len(unique_elements)} unique elements: {sorted(unique_elements)}")
    return unique_elements


def prompt_for_oxidation_states(elements):
    """
    Interactive prompt for oxidation state assignment.
    Shows smart defaults based on oxide chemistry, allows user override.
    
    Args:
        elements: set of element symbols
    
    Returns:
        dict mapping element → oxidation state {e.g., 'Ti': 4, 'Na': 1}
    """
    # Smart defaults for oxide battery materials AND halides/sulfides
    OXIDE_DEFAULTS = {
        # Alkali metals
        'Li': 1, 'Na': 1, 'K': 1, 'Rb': 1, 'Cs': 1,
        # Alkaline earth
        'Mg': 2, 'Ca': 2, 'Sr': 2, 'Ba': 2,
        # Transition metals (oxide battery cathodes)
        'Ti': 4,   # Ti⁴⁺ most common in oxides
        'V': 5,    # V⁵⁺ in V₂O₅
        'Cr': 3,   # Cr³⁺ in chromites
        'Mn': 4,   # Mn⁴⁺ in MnO₂, battery cathodes (can also be 2, 3)
        'Fe': 3,   # Fe³⁺ slightly more common (can also be 2)
        'Co': 3,   # Co³⁺ in LiCoO₂
        'Ni': 2,   # Ni²⁺ in most cathodes
        'Cu': 2,   # Cu²⁺ most common
        'Zn': 2,   # Zn²⁺ only
        'Cd': 2,   # Cd²⁺ only
        # Other common
        'Al': 3, 'Sb': 5, 'Sn': 4, 'Zr': 4, 'Nb': 5, 'Mo': 6, 'W': 6,
        # Anions (oxides, halides, chalcogenides)
        'O': -2, 'F': -1, 'Cl': -1, 'Br': -1, 'I': -1,
        'S': -2, 'Se': -2, 'Te': -2,
        'N': -3, 'P': -3, 'As': -3, 'H': -1,
    }
    
    print("\n" + "="*80)
    print("OXIDATION STATE ASSIGNMENT")
    print("="*80)
    print("Please confirm oxidation states for elements found in your CIF files.")
    print("Defaults are shown in [brackets] - press ENTER to accept, or type new value.")
    print("="*80 + "\n")
    
    oxidation_states = {}
    
    # Sort: anions last, then alphabetical
    anions = sorted([e for e in elements if e in ['O', 'F', 'S', 'Cl', 'Br', 'I']])
    cations = sorted([e for e in elements if e not in anions])
    sorted_elements = cations + anions
    
    for element in sorted_elements:
        # Get default suggestion
        default = OXIDE_DEFAULTS.get(element)
        
        if default is not None:
            prompt_text = f"  {element:>2s} oxidation state [{default:+d}]: "
        else:
            # Check database
            common_states = db_manager.get_common_oxidation_states(element)
            if common_states:
                # For elements with multiple common states, show options
                states_str = ', '.join([f"{s:+d}" for s in common_states])
                default = common_states[0]  # Suggest first
                prompt_text = f"  {element:>2s} oxidation state (common: {states_str}) [{default:+d}]: "
            else:
                prompt_text = f"  {element:>2s} oxidation state [?]: "
                default = None
        
        while True:
            try:
                user_input = input(prompt_text).strip()
                
                if user_input == '' and default is not None:
                    # Accept default
                    oxidation_states[element] = default
                    break
                elif user_input != '':
                    # Parse user input
                    valence = int(user_input)
                    oxidation_states[element] = valence
                    break
                else:
                    # No default and no input
                    print("    ERROR: Please enter an oxidation state")
            
            except ValueError:
                print("    ERROR: Please enter a valid integer (e.g., 4, -2)")
    
    # Show summary
    print("\n" + "="*80)
    print("OXIDATION STATES CONFIRMED:")
    print("="*80)
    for element in sorted_elements:
        valence = oxidation_states[element]
        print(f"  {element:>2s}: {valence:+d}")
    print("="*80 + "\n")
    
    # Final confirmation
    confirm = input("Proceed with these oxidation states? [Y/n]: ").strip().lower()
    if confirm and confirm not in ['y', 'yes', '']:
        print("\nAborted by user")
        return None
    
    return oxidation_states

def batch_process_directory(directory_path):
    """
    Process all CIF files in a directory using EXACT Skuld 1.5 logic.
    Includes interactive oxidation state checkpoint for user control.
    Automatically creates /outputs subfolder for results.
    
    Args:
        directory_path: Path to directory containing CIF files
    
    Returns:
        pandas DataFrame with results, or None if failed
    """
    # Create outputs subdirectory
    output_dir = os.path.join(directory_path, "outputs")
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n✓ Output directory: {output_dir}")
    
    cif_files = list(Path(directory_path).glob("*.cif"))
    
    if not cif_files:
        print(f"No CIF files found in {directory_path}")
        return None
    
    # ============================================================================
    # STEP 1: Try loading oxidation states from config file first
    # ============================================================================
    config_path = auto_detect_config_file(directory_path)
    user_oxidation_states = None
    
    if config_path:
        user_oxidation_states = load_oxidation_states_from_config(config_path=config_path)
    
    # ============================================================================
    # STEP 2: Fallback to interactive mode if no config file
    # ============================================================================
    if user_oxidation_states is None:
        print("No config file found or loading failed. Using interactive mode.")
        unique_elements = collect_unique_elements_from_cifs(directory_path)
        
        if not unique_elements:
            print("✗ No elements found in CIF files")
            return None
        
        user_oxidation_states = prompt_for_oxidation_states(unique_elements)
        
        if user_oxidation_states is None:
            print("\n✗ Batch processing cancelled by user")
            return None
    
    # ============================================================================
    # STEP 3: Process all CIF files with confirmed oxidation states
    # ============================================================================
    print(f"\n{'='*80}")
    print(f"EIR BATCH PROCESSOR v1.1 - Processing {len(cif_files)} CIF files")
    print(f"ICSD & GSAS-II compatible - validated Eir 1.0 logic")
    print(f"Using confirmed oxidation states")
    print(f"{'='*80}\n")
    
    all_results = []
    failed_files = []
    
    for i, cif_file in enumerate(cif_files, 1):
        print(f"[{i}/{len(cif_files)}] Processing {cif_file.name}...", end=" ")
        
        # Process with user-confirmed oxidation states
        result = process_single_cif(str(cif_file), user_oxidation_states)
        
        if result['success']:
            gii_str = f"{result['gii']:.4f}" if result['gii'] is not None else "N/A"
            print(f"✓ ({len(result['sites'])} sites, GII={gii_str})")
            
            # Add each site to results
            for site in result['sites']:
                row = {
                    **result['metadata'], 
                    **site,
                    'gii': round(result['gii'], 4) if result['gii'] is not None else None
                }
                all_results.append(row)
        else:
            print(f"✗ ({result['error']})")
            failed_files.append({
                'filename': cif_file.name,
                'error': result['error']
            })
    
    # ============================================================================
    # STEP 4: Save results to CSV files
    # ============================================================================
    if all_results:
        df = pd.DataFrame(all_results)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = os.path.join(output_dir, f"eir_batch_results_v1.2_{timestamp}.csv")
        df.to_csv(results_file, index=False)
        print(f"\n✓ Results saved to: {results_file}")
        
        # Save oxidation states used for this batch
        oxidation_states_file = os.path.join(output_dir, f"oxidation_states_used_{timestamp}.json")
        with open(oxidation_states_file, 'w') as f:
            import json
            json.dump(user_oxidation_states, f, indent=2, sort_keys=True)
        print(f"✓ Oxidation states saved to: {oxidation_states_file}")
        
        # Save failed files log if any
        if failed_files:
            failed_df = pd.DataFrame(failed_files)
            failed_file = os.path.join(output_dir, f"eir_batch_failed_v1.2_{timestamp}.csv")
            failed_df.to_csv(failed_file, index=False)
            print(f"✓ Failed files log saved to: {failed_file}")
            print(f"  ({len(failed_files)} files failed)")
        
        # Print summary statistics
        print(f"{'='*80}")
        print(f"EIR BATCH PROCESSOR v1.2 - Processing {len(cif_files)} CIF files")
        print(f"ICSD & GSAS-II compatible - Multi-anion support (O, F, Cl, Br, I, S, Se, Te)")
        print(f"Using confirmed oxidation states")
        print(f"{'='*80}\n")
        
        # Show oxidation states used
        print(f"\nOxidation states used:")
        for element in sorted(user_oxidation_states.keys()):
            valence = user_oxidation_states[element]
            print(f"  {element:>2s}: {valence:+d}")
        
        # Show GII statistics if available
        if 'gii' in df.columns and df['gii'].notna().any():
            print(f"\nGlobal Instability Index (GII) statistics:")
            gii_by_structure = df.groupby('filename')['gii'].first()
            print(f"  Mean GII: {gii_by_structure.mean():.4f}")
            print(f"  Min GII:  {gii_by_structure.min():.4f} ({gii_by_structure.idxmin()})")
            print(f"  Max GII:  {gii_by_structure.max():.4f} ({gii_by_structure.idxmax()})")
        
        print(f"{'='*80}\n")
        
        return df
    else:
        print("\n✗ No successful analyses")
        if failed_files:
            print(f"All {len(failed_files)} files failed. Check error messages above.")
        return None

# ============================================================================
# CONVENIENCE FUNCTIONS
# ============================================================================

def run_batch_analysis():
    """
    Convenience function for easy use in Jupyter notebooks or scripts.
    Opens directory selection dialog and processes all CIF files.
    
    Usage:
        from Eir_v1_1 import run_batch_analysis
        df = run_batch_analysis()
    
    Returns:
        pandas DataFrame with results, or None if failed/cancelled
    """
    print("\n" + "="*80)
    print("Running Eir v1.2 Batch Analysis...")
    print("="*80)
    
    # Select directory with tkinter dialog
    directory = select_directory()
    
    if directory is None:
        return None
    
    # Process all CIF files
    results_df = batch_process_directory(directory)
    
    if results_df is not None:
        print("\n" + "="*80)
        print("BATCH PROCESSING COMPLETE")
        print("="*80)
        print(f"Successfully analyzed: {results_df['filename'].nunique()} structures")
        print(f"Total sites analyzed: {len(results_df)}")
        output_dir = os.path.join(directory, "outputs")
        print(f"\nAll results saved to: {output_dir}")
        print("\nNext steps:")
        print("  1. Review the results CSV file")
        print("  2. Check for any failed files in the failed log")
        print("  3. Analyze results with statistical tools or custom scripts")
        
        return results_df
    
    return None

# ============================================================================
# EXECUTE BATCH ANALYSIS (Jupyter notebook mode)
# ============================================================================

# Run the batch analysis and store results in 'df' for Cell 5 to analyze
print("\n" + "="*80)
print("Running Eir v1.2 Batch Analysis...")
print("="*80)

df = run_batch_analysis()

if df is not None:
    print(f"\n✓ Analysis complete! Results stored in 'df' variable")
    print(f"✓ Run Cell 5 to see detailed analysis and problem site visualisation")
else:
    print(f"\n✗ Analysis cancelled or failed")

✓ pymatgen available - ICSD CIF files supported with proper crystallographic symmetry
✓ py3Dmol available - 3D visualisation enabled for problematic sites
✓ Loaded Shannon radii database with 441 entries
✓ Loaded BVS parameters database with 1384 entries
✓ Loaded oxidation states database with 80 elements
✓ Loaded element categories database

Running Eir v1.2 Batch Analysis...

Running Eir v1.2 Batch Analysis...

SELECT DIRECTORY CONTAINING CIF FILES
A file dialog will open. Please select the directory containing your CIF files.
(If the dialog appears behind other windows, check your taskbar)

✓ Selected directory: C:/Users/I6USER1/Downloads/Homovalent alkali metal halides
✓ Found 37 CIF files

✓ Output directory: C:/Users/I6USER1/Downloads/Homovalent alkali metal halides\outputs
No config file found or loading failed. Using interactive mode.

PRE-SCANNING CIF FILES TO IDENTIFY ELEMENTS
✓ Found 8 unique elements: ['Ag', 'Br', 'Cl', 'I', 'K', 'Na', 'Rb', 'Tl']

OXIDATION STATE ASSIGNMEN

  Ag oxidation state (common: +1) [+1]:  
   K oxidation state [+1]:  
  Na oxidation state [+1]:  
  Rb oxidation state [+1]:  
  Tl oxidation state (common: +1, +3) [+1]:  
  Br oxidation state [-1]:  
  Cl oxidation state [-1]:  
   I oxidation state [-1]:  



OXIDATION STATES CONFIRMED:
  Ag: +1
   K: +1
  Na: +1
  Rb: +1
  Tl: +1
  Br: -1
  Cl: -1
   I: -1



Proceed with these oxidation states? [Y/n]:  y



EIR BATCH PROCESSOR v1.1 - Processing 37 CIF files
ICSD & GSAS-II compatible - validated Eir 1.0 logic
Using confirmed oxidation states

[1/37] Processing Homovalent alkali metal halides_CollCode119597.cif...   ⚠ No bond data in CIF - calculating from crystallographic symmetry (ICSD mode)...
  Debug: Found 3 sites in asymmetric unit:
    K1: K at (0.0000, 0.0000, 0.0000) occ=1.00
    Cl1: Cl at (0.5000, 0.5000, 0.5000) occ=0.50
    Br1: Br at (0.5000, 0.5000, 0.5000) occ=0.50
  Generating 3×3×3 supercell for coordination analysis...
  Generated 15552 atoms in supercell
  Debug: Site K1 (K) has 6 neighbors in supercell
    -> Cl1 (Cl) dist=3.2250
    -> Cl1 (Cl) dist=3.2250
    -> Cl1 (Cl) dist=3.2250
    -> Cl1 (Cl) dist=3.2250
    -> Cl1 (Cl) dist=3.2250
    -> Cl1 (Cl) dist=3.2250
  Debug: Site Cl1 (Cl) has 6 neighbors in supercell
    -> K1 (K) dist=3.2250
    -> K1 (K) dist=3.2250
    -> K1 (K) dist=3.2250
    -> K1 (K) dist=3.2250
    -> K1 (K) dist=3.2250
    -> K1 (K) dist=3.22

...


✓ (1 sites, GII=0.3837)
[31/37] Processing Homovalent alkali metal halides_CollCode61666.cif...   ⚠ No bond data in CIF - calculating from crystallographic symmetry (ICSD mode)...
  Debug: Found 3 sites in asymmetric unit:
    Na1: Na at (0.0000, 0.0000, 0.0000) occ=1.00
    Br1: Br at (0.5000, 0.5000, 0.5000) occ=0.40
    Cl1: Cl at (0.5000, 0.5000, 0.5000) occ=0.60
  Generating 3×3×3 supercell for coordination analysis...
  Generated 15552 atoms in supercell
  Debug: Site Na1 (Na) has 6 neighbors in supercell
    -> Br1 (Br) dist=2.8910
    -> Br1 (Br) dist=2.8910
    -> Br1 (Br) dist=2.8910
    -> Br1 (Br) dist=2.8910
    -> Br1 (Br) dist=2.8910
    -> Br1 (Br) dist=2.8910
  Debug: Site Br1 (Br) has 6 neighbors in supercell
    -> Na1 (Na) dist=2.8910
    -> Na1 (Na) dist=2.8910
    -> Na1 (Na) dist=2.8910
    -> Na1 (Na) dist=2.8910
    -> Na1 (Na) dist=2.8910
    -> Na1 (Na) dist=2.8910
  Debug: Site Cl1 (Cl) has 6 neighbors in supercell
    -> Na1 (Na) dist=2.8910
    -> Na1 (Na

3Dmol.js failed to load for some reason. Please check your browser console for error messages.


✓ (1 sites, GII=0.3172)
[32/37] Processing Homovalent alkali metal halides_CollCode61667.cif...   ⚠ No bond data in CIF - calculating from crystallographic symmetry (ICSD mode)...
  Debug: Found 3 sites in asymmetric unit:
    Na1: Na at (0.0000, 0.0000, 0.0000) occ=1.00
    Br1: Br at (0.5000, 0.5000, 0.5000) occ=0.49
    Cl1: Cl at (0.5000, 0.5000, 0.5000) occ=0.51
  Generating 3×3×3 supercell for coordination analysis...
  Generated 15552 atoms in supercell
  Debug: Site Na1 (Na) has 6 neighbors in supercell
    -> Br1 (Br) dist=2.9063
    -> Br1 (Br) dist=2.9063
    -> Br1 (Br) dist=2.9063
    -> Br1 (Br) dist=2.9063
    -> Br1 (Br) dist=2.9063
    -> Br1 (Br) dist=2.9063
  Debug: Site Br1 (Br) has 6 neighbors in supercell
    -> Na1 (Na) dist=2.9063
    -> Na1 (Na) dist=2.9063
    -> Na1 (Na) dist=2.9063
    -> Na1 (Na) dist=2.9063
    -> Na1 (Na) dist=2.9063
    -> Na1 (Na) dist=2.9063
  Debug: Site Cl1 (Cl) has 6 neighbors in supercell
    -> Na1 (Na) dist=2.9063
    -> Na1 (Na

3Dmol.js failed to load for some reason. Please check your browser console for error messages.


✓ (2 sites, GII=0.1989)

✓ Results saved to: C:/Users/I6USER1/Downloads/Homovalent alkali metal halides\outputs\eir_batch_results_v1.2_20251211_164243.csv
✓ Oxidation states saved to: C:/Users/I6USER1/Downloads/Homovalent alkali metal halides\outputs\oxidation_states_used_20251211_164243.json
✓ Failed files log saved to: C:/Users/I6USER1/Downloads/Homovalent alkali metal halides\outputs\eir_batch_failed_v1.2_20251211_164243.csv
  (1 files failed)
EIR BATCH PROCESSOR v1.2 - Processing 37 CIF files
ICSD & GSAS-II compatible - Multi-anion support (O, F, Cl, Br, I, S, Se, Te)
Using confirmed oxidation states


Oxidation states used:
  Ag: +1
  Br: -1
  Cl: -1
   I: -1
   K: +1
  Na: +1
  Rb: +1
  Tl: +1

Global Instability Index (GII) statistics:
  Mean GII: 0.3464
  Min GII:  0.0597 (Homovalent alkali metal halides_CollCode61671.cif)
  Max GII:  0.7591 (Homovalent alkali metal halides_CollCode240594.cif)


BATCH PROCESSING COMPLETE
Successfully analyzed: 36 structures
Total sites analyze

In [109]:
# CELL 5: ENHANCED ANALYSIS WITH MEASUREMENT CONDITIONS
# ============================================================================
# Run this cell AFTER Cell 4 (which creates the 'df' variable)
# v1.2: Analysis includes measurement condition awareness

import pandas as pd
import numpy as np

# Check if df exists
if 'df' not in locals() or df is None:
    print("⚠️ ERROR: No results found!")
    print("Please run Cell 4 first to generate the 'df' DataFrame")
else:
    # ========================================================================
    # PART 0: MEASUREMENT CONDITIONS OVERVIEW
    # ========================================================================
    
    print("="*80)
    print("EIR v1.2 ANALYSIS WITH MEASUREMENT CONDITIONS")
    print("="*80)
    
    if 'measurement_conditions' in df.columns:
        print("\n🌡️  MEASUREMENT CONDITIONS DISTRIBUTION:")
        print("-"*80)
        
        # Get unique structures
        structures = df.groupby('filename')
        
        # Count by condition
        condition_summary = {}
        for filename, struct_data in structures:
            condition = struct_data['measurement_conditions'].iloc[0]
            if condition not in condition_summary:
                condition_summary[condition] = []
            condition_summary[condition].append(filename)
        
        # Print summary
        total_structures = len(df['filename'].unique())
        for condition in sorted(condition_summary.keys()):
            count = len(condition_summary[condition])
            percent = (count / total_structures) * 100
            print(f"  {condition:20s} n={count:3d} ({percent:5.1f}%)")
        
        # Highlight high-pressure structures
        high_p_conditions = [c for c in condition_summary.keys() if 'high_P' in c or 'high_T_high_P' in c]
        if high_p_conditions:
            total_high_p = sum(len(condition_summary[c]) for c in high_p_conditions)
            print(f"\n  ⚠️  High-pressure structures: n={total_high_p}")
            print(f"      These should be analyzed separately from ambient data!")
        
        # Show examples with temperature/pressure
        print("\n📊 SAMPLE STRUCTURES WITH METADATA:")
        print("-"*80)
        
        sample_conditions = ['ambient', 'high_T_high_P', 'high_P', 'unknown']
        for condition in sample_conditions:
            if condition in condition_summary:
                sample_file = condition_summary[condition][0]
                sample_data = df[df['filename'] == sample_file].iloc[0]
                
                temp = sample_data.get('measurement_temperature_K')
                pressure = sample_data.get('measurement_pressure_GPa')
                
                temp_str = f"{temp:.0f} K" if pd.notna(temp) else "not specified"
                pressure_str = f"{pressure:.2f} GPa" if pd.notna(pressure) else "not specified"
                
                print(f"\n  {condition}:")
                print(f"    File: {sample_file}")
                print(f"    Temperature: {temp_str}")
                print(f"    Pressure: {pressure_str}")
    
    else:
        print("\n⚠️  No measurement condition data found")
    
    # ========================================================================
    # PART 1: STATISTICAL OUTLIER DETECTION
    # ========================================================================
    
    print("\n\n" + "="*80)
    print("STATISTICAL OUTLIER ANALYSIS")
    print("="*80)
    
    # Calculate z-scores for key numerical columns
    df_outliers = df.copy()
    
    # For each element, calculate z-scores within that element group
    elements = df['element'].unique()
    
    for element in elements:
        element_mask = df['element'] == element
        element_data = df[element_mask]
        
        if len(element_data) > 1:  # Need at least 2 samples for std
            # BVS z-score
            if 'bvs' in df.columns:
                mean_bvs = element_data['bvs'].mean()
                std_bvs = element_data['bvs'].std()
                if std_bvs > 0:
                    df_outliers.loc[element_mask, 'bvs_zscore'] = (element_data['bvs'] - mean_bvs) / std_bvs
            
            # Deviation z-score
            if 'deviation' in df.columns:
                mean_dev = element_data['deviation'].mean()
                std_dev = element_data['deviation'].std()
                if std_dev > 0:
                    df_outliers.loc[element_mask, 'deviation_zscore'] = (element_data['deviation'] - mean_dev) / std_dev
            
            # Bond length z-score
            if 'avg_bond_length' in df.columns:
                mean_bl = element_data['avg_bond_length'].mean()
                std_bl = element_data['avg_bond_length'].std()
                if std_bl > 0:
                    df_outliers.loc[element_mask, 'bond_length_zscore'] = (element_data['avg_bond_length'] - mean_bl) / std_bl
    
    # Calculate GII z-scores across all structures
    if 'gii' in df.columns:
        mean_gii = df['gii'].mean()
        std_gii = df['gii'].std()
        if std_gii > 0:
            df_outliers['gii_zscore'] = (df['gii'] - mean_gii) / std_gii
    
    # Identify outliers (|z-score| > 2.0 = ~95% confidence)
    outlier_threshold = 2.0
    df_outliers['is_outlier'] = False
    
    for col in ['bvs_zscore', 'deviation_zscore', 'bond_length_zscore']:
        if col in df_outliers.columns:
            df_outliers['is_outlier'] |= (df_outliers[col].abs() > outlier_threshold)
    
    # Report outliers
    outliers = df_outliers[df_outliers['is_outlier'] == True]
    
    if len(outliers) > 0:
        print(f"\nFound {len(outliers)} outlier sites (|z-score| > {outlier_threshold}):")
        print()
        for idx, row in outliers.iterrows():
            print(f"  {row['filename'].replace('YourCustomFileName_', '')}")
            print(f"    Site: {row['element']} (label: {row['site_label']})")
            
            if 'bvs_zscore' in row and not pd.isna(row['bvs_zscore']):
                print(f"    BVS z-score: {row['bvs_zscore']:.2f} (BVS={row['bvs']:.3f}, expected={row['valence']:+.0f})")
            if 'deviation_zscore' in row and not pd.isna(row['deviation_zscore']):
                print(f"    Deviation z-score: {row['deviation_zscore']:.2f} (Δ={row['deviation']:+.3f} v.u.)")
            if 'bond_length_zscore' in row and not pd.isna(row['bond_length_zscore']):
                print(f"    Bond length z-score: {row['bond_length_zscore']:.2f} ({row['avg_bond_length']:.4f} Å)")
            
            # Show measurement conditions if available
            if 'measurement_conditions' in row:
                print(f"    Measurement: {row['measurement_conditions']}")
            
            print()
    else:
        print("\n✓ No statistical outliers detected (all z-scores < 2.0)")
    
    # ========================================================================
    # PART 2: QUALITY VERDICTS
    # ========================================================================
    
    print("\n" + "="*80)
    print("QUALITY ASSESSMENT & VERDICTS")
    print("="*80)
    
    # Get unique structures
    structures = df.groupby('filename')
    
    quality_verdicts = []
    
    for filename, struct_data in structures:
        verdict = {
            'filename': filename,
            'icsd_code': struct_data['block_name'].iloc[0] if 'block_name' in struct_data.columns else None,
            'gii': struct_data['gii'].iloc[0],
            'n_sites': len(struct_data),
            'n_problematic': len(struct_data[struct_data['deviation'].abs() > 0.30]),
            'max_deviation': struct_data['deviation'].abs().max(),
            'conditions': struct_data['measurement_conditions'].iloc[0] if 'measurement_conditions' in struct_data.columns else 'unknown',
            'flags': [],
            'verdict': 'UNKNOWN',
            'quality_tier': 'UNKNOWN'
        }
        
        # BVS Check
        if verdict['max_deviation'] > 0.30:
            verdict['flags'].append('BVS_FAIL')
        elif verdict['max_deviation'] > 0.20:
            verdict['flags'].append('BVS_WARN')
        else:
            verdict['flags'].append('BVS_PASS')
        
        # GII Check
        if verdict['gii'] > 0.40:
            verdict['flags'].append('GII_FAIL')
        elif verdict['gii'] > 0.20:
            verdict['flags'].append('GII_WARN')
        else:
            verdict['flags'].append('GII_PASS')
        
        # Check for statistical outliers
        struct_outliers = df_outliers[df_outliers['filename'] == filename]
        if struct_outliers['is_outlier'].any():
            verdict['flags'].append('STATISTICAL_OUTLIER')
        
        # Check for non-ambient conditions
        if 'high_P' in verdict['conditions'] or 'high_T' in verdict['conditions']:
            verdict['flags'].append('NON_AMBIENT')
        
        # Overall verdict
        if 'NON_AMBIENT' in verdict['flags']:
            verdict['verdict'] = '⚠️ NON-AMBIENT'
            verdict['quality_tier'] = 'Exclude from ambient analysis'
        elif 'BVS_FAIL' in verdict['flags'] or 'GII_FAIL' in verdict['flags']:
            verdict['verdict'] = '⛔ EXCLUDE'
            verdict['quality_tier'] = 'F (Reject)'
        elif 'STATISTICAL_OUTLIER' in verdict['flags']:
            verdict['verdict'] = '⚠️ QUESTIONABLE'
            verdict['quality_tier'] = 'C (Suspicious)'
        elif 'BVS_WARN' in verdict['flags'] or 'GII_WARN' in verdict['flags']:
            verdict['verdict'] = '⚠️ CAUTION'
            verdict['quality_tier'] = 'B (Use with care)'
        else:
            verdict['verdict'] = '✅ INCLUDE'
            verdict['quality_tier'] = 'A (High confidence)'
        
        quality_verdicts.append(verdict)
    
    # Display verdicts
    print()
    for v in sorted(quality_verdicts, key=lambda x: x['gii'], reverse=True):
        print(f"{v['verdict']} {v['filename'].replace('YourCustomFileName_', '')}")
        print(f"  Quality Tier: {v['quality_tier']}")
        print(f"  Conditions: {v['conditions']}")
        print(f"  GII: {v['gii']:.4f}  │  Max |Δ|: {v['max_deviation']:.3f} v.u.")
        print(f"  Problematic sites: {v['n_problematic']}/{v['n_sites']}")
        print(f"  Flags: {', '.join(v['flags'])}")
        print()
    
    # ========================================================================
    # PART 3: COMPARATIVE STATISTICS BY ELEMENT (CONDITION-AWARE)
    # ========================================================================
    
    print("="*80)
    print("COMPARATIVE STATISTICS (By Element, Ambient vs High-P)")
    print("="*80)
    print()
    
    # Separate ambient from non-ambient
    if 'measurement_conditions' in df.columns:
        ambient_df = df[df['measurement_conditions'].isin(['ambient', 'unknown'])]
        high_p_df = df[df['measurement_conditions'].str.contains('high_P', na=False)]
        
        if len(ambient_df) > 0:
            print("AMBIENT STRUCTURES:")
            print("-"*80)
            
            # Group by element
            element_stats = []
            
            for element in sorted(ambient_df['element'].unique()):
                element_data = ambient_df[ambient_df['element'] == element]
                
                # Exclude outliers for "consensus" values
                if element in df_outliers['element'].values:
                    element_outliers = df_outliers[(df_outliers['element'] == element) & (df_outliers['is_outlier'] == True)]
                    element_consensus = element_data[~element_data.index.isin(element_outliers.index)]
                else:
                    element_consensus = element_data
                
                if len(element_consensus) > 0:
                    stats = {
                        'element': element,
                        'valence': element_data['valence'].iloc[0],
                        'n_total': len(element_data),
                        'n_outliers': len(element_data) - len(element_consensus),
                        'bvs_mean': element_consensus['bvs'].mean(),
                        'bvs_std': element_consensus['bvs'].std(),
                        'deviation_mean': element_consensus['deviation'].mean(),
                        'deviation_std': element_consensus['deviation'].std(),
                        'bond_length_mean': element_consensus['avg_bond_length'].mean(),
                        'bond_length_std': element_consensus['avg_bond_length'].std(),
                        'cn_mode': element_consensus['cn'].mode()[0] if len(element_consensus['cn'].mode()) > 0 else None
                    }
                    
                    element_stats.append(stats)
            
            # Display as table
            if element_stats:
                stats_df = pd.DataFrame(element_stats)
                
                print("Element | Valence | n | BVS (consensus) | Deviation | Bond Length (Å) | CN")
                print("-"*80)
                for _, row in stats_df.iterrows():
                    bvs_str = f"{row['bvs_mean']:.3f}±{row['bvs_std']:.3f}" if pd.notna(row['bvs_std']) else f"{row['bvs_mean']:.3f}"
                    dev_str = f"{row['deviation_mean']:+.3f}±{row['deviation_std']:.3f}" if pd.notna(row['deviation_std']) else f"{row['deviation_mean']:+.3f}"
                    bl_str = f"{row['bond_length_mean']:.4f}±{row['bond_length_std']:.4f}" if pd.notna(row['bond_length_std']) else f"{row['bond_length_mean']:.4f}"
                    
                    outlier_note = f" ({row['n_outliers']} outlier)" if row['n_outliers'] > 0 else ""
                    
                    print(f"{row['element']:>7} | {row['valence']:>7.0f} | {row['n_total']:>1}{outlier_note:<12} | {bvs_str:<15} | {dev_str:<9} | {bl_str:<15} | {row['cn_mode']}")
        
        if len(high_p_df) > 0:
            print("\n\nHIGH-PRESSURE STRUCTURES:")
            print("-"*80)
            print("⚠️  These structures should be analyzed separately!")
            print("    BVS parameters are fitted to ambient conditions.\n")
            
            # Group by element
            for element in sorted(high_p_df['element'].unique()):
                element_data = high_p_df[high_p_df['element'] == element]
                
                print(f"\n{element} (n={len(element_data)}):")
                print(f"  Mean deviation: {element_data['deviation'].mean():+.3f} ± {element_data['deviation'].std():.3f} v.u.")
                print(f"  Range: {element_data['deviation'].min():+.3f} to {element_data['deviation'].max():+.3f} v.u.")
                
                # Show pressure range
                if 'measurement_pressure_GPa' in element_data.columns:
                    pressures = element_data['measurement_pressure_GPa'].dropna()
                    if len(pressures) > 0:
                        print(f"  Pressure range: {pressures.min():.1f} - {pressures.max():.1f} GPa")
    
    else:
        # Fall back to original analysis if no condition data
        print("(No measurement condition data - showing all structures combined)")
        
        element_stats = []
        
        for element in sorted(elements):
            element_data = df[df['element'] == element]
            
            # Exclude outliers for "consensus" values
            if element in df_outliers['element'].values:
                element_outliers = df_outliers[(df_outliers['element'] == element) & (df_outliers['is_outlier'] == True)]
                element_consensus = element_data[~element_data.index.isin(element_outliers.index)]
            else:
                element_consensus = element_data
            
            stats = {
                'element': element,
                'valence': element_data['valence'].iloc[0],
                'n_total': len(element_data),
                'n_outliers': len(element_data) - len(element_consensus),
                'bvs_mean': element_consensus['bvs'].mean(),
                'bvs_std': element_consensus['bvs'].std(),
                'deviation_mean': element_consensus['deviation'].mean(),
                'deviation_std': element_consensus['deviation'].std(),
                'bond_length_mean': element_consensus['avg_bond_length'].mean(),
                'bond_length_std': element_consensus['avg_bond_length'].std(),
                'cn_mode': element_consensus['cn'].mode()[0] if len(element_consensus['cn'].mode()) > 0 else None
            }
            
            element_stats.append(stats)
        
        # Display as table
        stats_df = pd.DataFrame(element_stats)
        
        print("Element | Valence | n | BVS (consensus) | Deviation | Bond Length (Å) | CN")
        print("-"*80)
        for _, row in stats_df.iterrows():
            bvs_str = f"{row['bvs_mean']:.3f}±{row['bvs_std']:.3f}" if pd.notna(row['bvs_std']) else f"{row['bvs_mean']:.3f}"
            dev_str = f"{row['deviation_mean']:+.3f}±{row['deviation_std']:.3f}" if pd.notna(row['deviation_std']) else f"{row['deviation_mean']:+.3f}"
            bl_str = f"{row['bond_length_mean']:.4f}±{row['bond_length_std']:.4f}" if pd.notna(row['bond_length_std']) else f"{row['bond_length_mean']:.4f}"
            
            outlier_note = f" ({row['n_outliers']} outlier)" if row['n_outliers'] > 0 else ""
            
            print(f"{row['element']:>7} | {row['valence']:>7.0f} | {row['n_total']:>1}{outlier_note:<12} | {bvs_str:<15} | {dev_str:<9} | {bl_str:<15} | {row['cn_mode']}")
    
    # ========================================================================
    # PART 4: PUBLICATION METADATA SUMMARY
    # ========================================================================
    
    if 'year' in df.columns and df['year'].notna().any():
        print("\n" + "="*80)
        print("PUBLICATION METADATA")
        print("="*80)
        print()
        
        for filename in df['filename'].unique():
            struct = df[df['filename'] == filename].iloc[0]
            
            print(f"{filename.replace('YourCustomFileName_', '')}")
            if pd.notna(struct.get('icsd_code')):
                print(f"  ICSD Code: {struct['icsd_code']}")
            if pd.notna(struct.get('year')):
                print(f"  Year: {struct['year']}")
            if pd.notna(struct.get('journal')):
                print(f"  Journal: {struct['journal']}")
            if pd.notna(struct.get('authors')):
                print(f"  Authors: {struct['authors']}")
            if pd.notna(struct.get('r_factor')):
                print(f"  R-factor: {struct['r_factor']:.4f}")
            if pd.notna(struct.get('density')):
                print(f"  Density: {struct['density']:.2f} g/cm³")
            
            # Show measurement conditions
            if 'measurement_temperature_K' in struct and pd.notna(struct['measurement_temperature_K']):
                print(f"  Temperature: {struct['measurement_temperature_K']:.0f} K")
            if 'measurement_pressure_GPa' in struct and pd.notna(struct['measurement_pressure_GPa']):
                print(f"  Pressure: {struct['measurement_pressure_GPa']:.2f} GPa")
            if 'measurement_conditions' in struct:
                print(f"  Conditions: {struct['measurement_conditions']}")
            
            print()
    
    # ========================================================================
    # PART 5: PROBLEMATIC SITES
    # ========================================================================
    
    print("="*80)
    print("PROBLEMATIC SITES SUMMARY (|Deviation| > 0.30 v.u.)")
    print("="*80)
    
    problematic = df[df['deviation'].abs() > 0.30].sort_values('deviation', key=abs, ascending=False)
    
    if len(problematic) > 0:
        print(f"Found {len(problematic)} sites with significant BVS deviations:")
        for _, row in problematic.iterrows():
            icsd_id = row['block_name'].split('-')[0] if '-' in row['block_name'] else row['block_name']
            print(f"\n  {icsd_id}")
            print(f"    Site: {row['element']} (label: {row['site_label']})")
            print(f"    BVS: {row['bvs']:.3f}  │  Expected: {row['valence']:+.0f}  │  Δ: {row['deviation']:+.3f} v.u.")
            print(f"    CN: {row['cn']}")
            
            # Show measurement conditions
            if 'measurement_conditions' in row:
                print(f"    Conditions: {row['measurement_conditions']}")
            
            # Add z-score if available
            if row['element'] in df_outliers['element'].values:
                row_outlier = df_outliers[df_outliers.index == row.name]
                if not row_outlier.empty and 'deviation_zscore' in row_outlier.columns:
                    zscore = row_outlier['deviation_zscore'].iloc[0]
                    if pd.notna(zscore):
                        print(f"    Statistical: z-score = {zscore:.2f}")
            
            print(f"    [3D visualisation displayed during Cell 4 processing]")
    else:
        print("No sites with |Δ| > 0.30 v.u. found")
    
    print("\n" + "="*80)
    print("EIR v1.2 ENHANCED ANALYSIS COMPLETE")
    print("="*80)
    print("\nKey Outputs:")
    print("  1. Measurement conditions distribution")
    print("  2. Ambient vs high-pressure separation")
    print("  3. Statistical outlier detection with z-scores")
    print("  4. Quality verdicts with condition awareness")
    print("  5. Consensus statistics by element (outliers excluded)")
    print("  6. Publication metadata with temperature/pressure")
    print("  7. Problematic sites with conditions context")
    print("\nRecommended Actions:")
    print("  • High-P structures marked for separate analysis")
    print("  • Structures marked ⛔ EXCLUDE should be rejected from database")
    print("  • Structures marked ⚠️ QUESTIONABLE need manual inspection")
    print("  • Use ambient statistics for 'expected' values in publications")
    print("="*80)

EIR v1.2 ANALYSIS WITH MEASUREMENT CONDITIONS

🌡️  MEASUREMENT CONDITIONS DISTRIBUTION:
--------------------------------------------------------------------------------
  ambient              n=  4 ( 11.1%)
  unknown              n= 32 ( 88.9%)

📊 SAMPLE STRUCTURES WITH METADATA:
--------------------------------------------------------------------------------

  ambient:
    File: Homovalent alkali metal halides_CollCode240531.cif
    Temperature: 298 K
    Pressure: 0.00 GPa

  unknown:
    File: Homovalent alkali metal halides_CollCode119597.cif
    Temperature: not specified
    Pressure: not specified


STATISTICAL OUTLIER ANALYSIS

Found 3 outlier sites (|z-score| > 2.0):

  Homovalent alkali metal halides_CollCode22172.cif
    Site: K (label: K1)
    BVS z-score: -1.52 (BVS=0.737, expected=+1)
    Deviation z-score: -1.52 (Δ=-0.263 v.u.)
    Bond length z-score: 2.28 (3.4358 Å)
    Measurement: unknown

  Homovalent alkali metal halides_CollCode240594.cif
    Site: K (label: K1)
